# Description: 
The change detection conducted with Sentinel-2 satellite images on Dongting Lake, China, across wet and dry season. To find out how the hydrological change impacted surroundings. Besides, two methods: Band Math and Supervised Classification are applied, some quantitative comparisons are performed.

# Preparation:
Import Necessary Libraries for Google Earth Engine

In [ ]:
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize()

In [ ]:
import geemap
Map = geemap.Map()
Map

Get RGB composites of wet and dry seasons respectively

In [ ]:
roi = ee.Geometry.Polygon([[112.998596, 29.540793],
   [112.657961, 29.562296],
   [112.649719, 29.004124],
   [112.027511, 29.103764],
   [111.839338, 28.84426],
   [112.976619, 28.675716],
   [113.230721, 29.453537],
   [112.998596, 29.540793]])

#Get image of wet season
collection_wet = ee.ImageCollection('COPERNICUS/S2_SR') \
                       .filterDate('2019-06-01', '2019-09-01') \
                       .filterBounds(roi) \
                       .filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',1) \
                       .sort('CLOUD_PIXEL_PERCENTAGE')\
                       
image_wet = collection_wet.median().clip(roi)

#Get image of dry season
collection_dry = ee.ImageCollection('COPERNICUS/S2_SR') \
                       .filterDate('2019-11-01', '2020-03-01') \
                       .filterBounds(roi) \
                       .filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than',5) \
                       .sort('CLOUD_PIXEL_PERCENTAGE') \
                      
image_dry = collection_dry.median().clip(roi)

vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0.0, 'max': 2100, 'opacity': 1.0, 'gamma': 1.0}
                  
Map.setCenter(112.889291, 28.914755, 10)

Map.addLayer(image_wet, vis_params, "Wet image")
Map.addLayer(image_dry, vis_params, "Dry image")


# Mehtod 1: Band Math Change Detection: 
Calculate NDWI for images separately, NDWI = (B3-B5)/(B3+B5)

In [ ]:
NDWI_wet = image_wet.expression('(Green - NIR) / (Green + NIR)', {
      'Green': image_wet.select('B3'),  
      'NIR': image_wet.select('B8')
})

NDWI_dry = image_dry.expression('(Green - NIR) / (Green + NIR)', {
      'Green': image_dry.select('B3'),  
      'NIR': image_dry.select('B8')
})

#water body with binary value(0,1) by filtring value
water_wet = NDWI_wet.gt(0.3)
water_dry = NDWI_dry.gt(0.3)

#Water body extraction(for area calculation)
waterbody_wet = NDWI_wet.gt(0.3).selfMask()
waterbody_dry = NDWI_dry.gt(0.3).selfMask()

#Change Detection(Band Subtraction)
NDWI_Diff = water_wet.subtract(water_dry)

Map.addLayer(NDWI_wet, {'min': -1, 'max': 1, 'palette': ['white', 'black']}, 'NDWI_wet')
Map.addLayer(NDWI_dry, {'min': -1, 'max': 1, 'palette': ['white', 'black']}, 'NDWI_dry')
Map.addLayer(water_wet, {'min': 0, 'max': 1, 'palette': [ 'red', 'black']}, 'water_wet')
Map.addLayer(water_dry, {'min': 0, 'max': 1, 'palette': [ 'red', 'black']}, 'water_dry')
# Map.addLayer(waterbody_wet, {'min': 0, 'max': 1, 'palette': [ 'red']}, 'water_wet')
# Map.addLayer(waterbody_dry, {'min': 0, 'max': 1, 'palette': [ 'red']}, 'water_dry')
Map.addLayer(NDWI_Diff, {'min': 0, 'max': 1, 'palette': ['white', 'blue']}, 'NDWI_Diff')
Map

Area Calculation (Band math)

In [ ]:
#Wet season waterbody area (based on NDWI)
areaImage = waterbody_wet.multiply(ee.Image.pixelArea())
area_dic = areaImage.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': roi,
  'scale': 10,
  'maxPixels': 1e9
  })
area = area_dic.getInfo()['B3']
waterArea_wet = round(area/1e6)

#Dry season waterbody area
areaImage = waterbody_dry.multiply(ee.Image.pixelArea())
area_dic = areaImage.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': roi,
  'scale': 10,
  'maxPixels': 1e9
  })
area = area_dic.getInfo()['B3']
waterArea_dry = round(area/1e6)

#Area Difference between Dry and Wet season
areaImage = NDWI_Diff.multiply(ee.Image.pixelArea())
area_dic = areaImage.reduceRegion(**{
  'reducer': ee.Reducer.sum(),
  'geometry': roi,
  'scale': 10,
  'maxPixels': 1e9
  })
area = area_dic.getInfo()['B3']
waterArea_Diff = round(area/1e6)

print('Wet season area:', waterArea_wet)
print('Dry season area:', waterArea_dry)
print('Difference Area:', waterArea_Diff)
print('Region of Interest:', roi.area().divide(1e6).getInfo())

# Method 2: Supervised Classification & Error Assessment


Train rois for wet season

In [ ]:
# Use these bands for prediction.
bands = ['B2','B3','B4','B7','B8']

# Create trainning polygons.
water_body1 = ee.Geometry.Polygon([[113.006112, 29.239048],
   [113.006112, 29.257321],
   [113.027401, 29.257321],
   [113.027401, 29.239048],
   [113.006112, 29.239048]])
water_body2 = ee.Geometry.Polygon([[112.811071, 29.215379],
   [112.811071, 29.24444],
   [112.843692, 29.24444],
   [112.843692, 29.215379],
   [112.811071, 29.215379]])
water_body3 = ee.Geometry.Polygon([[112.772556, 28.967707],
   [112.772556, 28.969547],
   [112.777492, 28.969547],
   [112.777492, 28.967707],
   [112.772556, 28.967707]])
water_body4 = ee.Geometry.Polygon([[112.841973, 28.886753],
   [112.841973, 28.897273],
   [112.854506, 28.897273],
   [112.854506, 28.886753],
   [112.841973, 28.886753]])
water_body5 = ee.Geometry.Polygon([[112.758702, 28.792169],
   [112.758702, 28.805407],
   [112.777417, 28.805407],
   [112.777417, 28.792169],
   [112.758702, 28.792169]])
water_body6 = ee.Geometry.Polygon([[112.674918, 28.826464],
   [112.674918, 28.836089],
   [112.68831, 28.836089],
   [112.68831, 28.826464],
   [112.674918, 28.826464]])
water_body7 = ee.Geometry.Polygon([[112.430135, 28.811885],
   [112.430049, 28.7994],
   [112.438633, 28.793984],
   [112.452798, 28.791728],
   [112.457777, 28.798347],
   [112.435028, 28.817976],
   [112.430135, 28.811885]])
water_body8 = ee.Geometry.Polygon([[112.26525, 28.840684],
   [112.267826, 28.832113],
   [112.282334, 28.828353],
   [112.289029, 28.835496],
   [112.286969, 28.840308],
   [112.271002, 28.844519],
   [112.26525, 28.840684]])
water_body9 = ee.Geometry.Polygon([[112.176243, 28.840384],
   [112.179935, 28.834218],
   [112.189807, 28.830835],
   [112.196503, 28.834745],
   [112.200194, 28.841135],
   [112.201825, 28.845045],
   [112.190923, 28.846699],
   [112.186373, 28.841361],
   [112.176243, 28.840384]])
water_body10 = ee.Geometry.Polygon([[112.207338, 28.93057],
   [112.210063, 28.931415],
   [112.212467, 28.933969],
   [112.213626, 28.935621],
   [112.212725, 28.936166],
   [112.210493, 28.933218],
   [112.207338, 28.931565],
   [112.206157, 28.931678],
   [112.204612, 28.933556],
   [112.204355, 28.936466],
   [112.203947, 28.938213],
   [112.202445, 28.937762],
   [112.203561, 28.934607],
   [112.204934, 28.931603],
   [112.207338, 28.93057]])
water_body11 = ee.Geometry.Polygon([[112.197416, 28.962322],
   [112.194357, 28.959309],
   [112.190559, 28.956267],
   [112.191428, 28.955169],
   [112.195055, 28.958858],
   [112.198177, 28.961365],
   [112.197416, 28.962322]])
water_body12 = ee.Geometry.Polygon([[112.150882, 28.902733],
   [112.141912, 28.89954],
   [112.14247, 28.897999],
   [112.142985, 28.895895],
   [112.143457, 28.894317],
   [112.148436, 28.895031],
   [112.15848, 28.895219],
   [112.161055, 28.893415],
   [112.169082, 28.891837],
   [112.17097, 28.895369],
   [112.171485, 28.898976],
   [112.16436, 28.900629],
   [112.155947, 28.903184],
   [112.150882, 28.902733]])
water_body13 = ee.Geometry.Polygon([[112.254402, 28.927376],
   [112.251612, 28.923657],
   [112.253372, 28.91637],
   [112.255861, 28.913853],
   [112.259081, 28.910096],
   [112.263673, 28.911299],
   [112.267322, 28.914604],
   [112.270026, 28.920202],
   [112.263287, 28.918774],
   [112.258222, 28.920352],
   [112.258952, 28.923207],
   [112.256849, 28.926324],
   [112.254402, 28.927376]])
water_body14 = ee.Geometry.Polygon([[112.320998, 29.017558],
   [112.325419, 29.017295],
   [112.325118, 29.011516],
   [112.325977, 29.002733],
   [112.329153, 28.99166],
   [112.331385, 28.984378],
   [112.327865, 28.983214],
   [112.32426, 28.991322],
   [112.322114, 29.00217],
   [112.322071, 29.00735],
   [112.320611, 29.008326],
   [112.320998, 29.017558]])
water_body15 = ee.Geometry.Polygon([[112.323872, 29.044687],
   [112.331298, 29.048364],
   [112.338681, 29.039697],
   [112.343488, 29.037784],
   [112.343188, 29.035758],
   [112.337393, 29.036696],
   [112.335934, 29.035795],
   [112.335676, 29.031593],
   [112.333101, 29.031968],
   [112.333444, 29.037183],
   [112.331298, 29.04041],
   [112.327864, 29.042061],
   [112.326319, 29.044125],
   [112.323872, 29.044687]])
water_body16 = ee.Geometry.Polygon([[112.931557, 29.507142],
   [112.940141, 29.508562],
   [112.943404, 29.493322],
   [112.946923, 29.483909],
   [112.951645, 29.477035],
   [112.956321, 29.475356],
   [112.962202, 29.474235],
   [112.96688, 29.469677],
   [112.960656, 29.466576],
   [112.950698, 29.467995],
   [112.94632, 29.471097],
   [112.942028, 29.474534],
   [112.938379, 29.477822],
   [112.935976, 29.479577],
   [112.933528, 29.485702],
   [112.931897, 29.495638],
   [112.931557, 29.507142]])
water_body17 = ee.Geometry.Polygon([[112.861518, 29.411171],
   [112.861518, 29.443466],
   [112.905471, 29.443466],
   [112.905471, 29.411171],
   [112.861518, 29.411171]])
water_body18 = ee.Geometry.Polygon([[112.964563, 28.958359],
   [112.964563, 28.963766],
   [112.977439, 28.963766],
   [112.977439, 28.958359],
   [112.964563, 28.958359]])
water_body19 = ee.Geometry.Polygon([[112.90272, 29.002881],
   [112.90272, 29.017818],
   [112.924267, 29.017818],
   [112.924267, 29.002881],
   [112.90272, 29.002881]])
water_body20 = ee.Geometry.Polygon([[113.122362, 29.333884],
   [113.122362, 29.340057],
   [113.130131, 29.340057],
   [113.130131, 29.333884],
   [113.122362, 29.333884]])
water_body21 = ee.Geometry.Polygon([[113.003304, 29.31618],
   [113.003304, 29.343716],
   [113.047257, 29.343716],
   [113.047257, 29.31618],
   [113.003304, 29.31618]])
water_body22 = ee.Geometry.Polygon([[112.869631, 28.776536],
   [112.869631, 28.782686],
   [112.880662, 28.782686],
   [112.880662, 28.776536],
   [112.869631, 28.776536]])
water_body23 = ee.Geometry.Polygon([[112.835342, 28.796993],
   [112.835342, 28.813163],
   [112.855258, 28.813163],
   [112.855258, 28.796993],
   [112.835342, 28.796993]])
water_body24 = ee.Geometry.Polygon([[112.680625, 28.897827],
   [112.680502, 28.896498],
   [112.686468, 28.895827],
   [112.686731, 28.897207],
   [112.680625, 28.897827]])

#Nonwater Areas
cropland1 = ee.Geometry.Polygon([[112.92074, 28.939345],
   [112.92074, 28.960675],
   [112.944949, 28.960675],
   [112.944949, 28.939345],
   [112.92074, 28.939345]])
cropland2 = ee.Geometry.Polygon([[112.768205, 28.920873],
   [112.768205, 28.930939],
   [112.781855, 28.930939],
   [112.781855, 28.920873],
   [112.768205, 28.920873]])
cropland3 = ee.Geometry.Polygon([[112.697754, 28.90848],
   [112.697754, 28.916031],
   [112.710459, 28.916031],
   [112.710459, 28.90848],
   [112.697754, 28.90848]])
cropland4 = ee.Geometry.Polygon([[112.143705, 28.988227],
   [112.147654, 28.987664],
   [112.14687, 28.982202],
   [112.142427, 28.982783],
   [112.143705, 28.988227]])
cropland5 = ee.Geometry.Polygon([[112.13979, 28.99427],
   [112.143889, 28.993482],
   [112.14346, 28.991061],
   [112.139146, 28.991849],
   [112.13979, 28.99427]])
cropland6 = ee.Geometry.Polygon([[112.139962, 28.998437],
   [112.14434, 28.997517],
   [112.143868, 28.99504],
   [112.139275, 28.995772],
   [112.139962, 28.998437]])
cropland7 = ee.Geometry.Polygon([[112.173398, 28.998418],
   [112.184945, 28.996053],
   [112.185524, 28.993032],
   [112.187005, 28.99153],
   [112.186082, 28.987194],
   [112.173313, 28.989672],
   [112.174214, 28.994965],
   [112.172132, 28.995715],
   [112.173398, 28.998418]])
cropland8 = ee.Geometry.Polygon([[112.186213, 29.001647],
   [112.191546, 29.004377],
   [112.192566, 29.004471],
   [112.193853, 29.002782],
   [112.194733, 29.001797],
   [112.194959, 29.001356],
   [112.187705, 28.998456],
   [112.186213, 29.001647]])
cropland9 = ee.Geometry.Polygon([[112.724363, 28.998152],
   [112.760074, 28.992071],
   [112.758872, 28.986515],
   [112.754408, 28.985614],
   [112.723161, 28.991395],
   [112.724363, 28.998152]])
cropland10 = ee.Geometry.Polygon([[112.749133, 29.260978],
   [112.76128, 29.262214],
   [112.764242, 29.256523],
   [112.766602, 29.253153],
   [112.764542, 29.250607],
   [112.760936, 29.250495],
   [112.750206, 29.249109],
   [112.749133, 29.260978]])
cropland11 = ee.Geometry.Polygon([[112.714446, 29.237389],
   [112.724146, 29.237501],
   [112.724704, 29.231771],
   [112.715261, 29.230386],
   [112.714446, 29.237389]])
cropland12 = ee.Geometry.Polygon([[112.71432, 29.248184],
   [112.719889, 29.248737],
   [112.722325, 29.247361],
   [112.722454, 29.245329],
   [112.721381, 29.245189],
   [112.720705, 29.2462],
   [112.715876, 29.245853],
   [112.714846, 29.246846],
   [112.714105, 29.24693],
   [112.71432, 29.248184]])
cropland13 = ee.Geometry.Polygon([[112.724532, 29.2674],
   [112.733804, 29.268299],
   [112.734963, 29.261073],
   [112.725734, 29.260062],
   [112.724532, 29.2674]])
cropland14 = ee.Geometry.Polygon([[112.782041, 29.290833],
   [112.787578, 29.291376],
   [112.786741, 29.288288],
   [112.785967, 29.28805],
   [112.785763, 29.287419],
   [112.78219, 29.287105],
   [112.782041, 29.290833]])
cropland15 = ee.Geometry.Polygon([[112.767818, 29.275192],
   [112.768704, 29.275281],
   [112.768779, 29.274271],
   [112.767931, 29.274065],
   [112.767818, 29.275192]])
cropland16 = ee.Geometry.Polygon([[112.764707, 29.279563],
   [112.765748, 29.279558],
   [112.765785, 29.278931],
   [112.765147, 29.278856],
   [112.765206, 29.277902],
   [112.764905, 29.277972],
   [112.764798, 29.278664],
   [112.764707, 29.279563]])
cropland17 = ee.Geometry.Polygon([[112.77218, 29.289866],
   [112.772818, 29.289875],
   [112.773269, 29.286478],
   [112.771128, 29.286197],
   [112.770887, 29.287821],
   [112.772357, 29.288111],
   [112.77218, 29.289866]])
cropland18 = ee.Geometry.Polygon([[112.76438, 29.301874],
   [112.767245, 29.302034],
   [112.767277, 29.30107],
   [112.768318, 29.301266],
   [112.768565, 29.298983],
   [112.766354, 29.298703],
   [112.766215, 29.300256],
   [112.764541, 29.300247],
   [112.76438, 29.301874]])
cropland19 = ee.Geometry.Polygon([[112.762266, 29.317367],
   [112.764004, 29.317423],
   [112.764187, 29.315496],
   [112.765313, 29.31558],
   [112.765785, 29.312381],
   [112.762931, 29.312063],
   [112.762266, 29.317367]])
cropland20 = ee.Geometry.Polygon([[112.687336, 28.963627],
   [112.691371, 28.962078],
   [112.689342, 28.957891],
   [112.685887, 28.959562],
   [112.687336, 28.963627]])
cropland21 = ee.Geometry.Polygon([[112.654978, 28.934674],
   [112.659957, 28.933556],
   [112.659431, 28.931735],
   [112.655836, 28.932458],
   [112.656072, 28.933669],
   [112.654549, 28.934016],
   [112.654978, 28.934674]])
cropland22 = ee.Geometry.Polygon([[112.400998, 28.96861],
   [112.409325, 28.964555],
   [112.404174, 28.954942],
   [112.394645, 28.95911],
   [112.400998, 28.96861]])
cropland23 = ee.Geometry.Polygon([[112.457346, 28.906527],
   [112.465973, 28.907805],
   [112.468849, 28.900779],
   [112.471983, 28.899164],
   [112.473099, 28.894054],
   [112.463441, 28.893753],
   [112.458805, 28.897248],
   [112.457346, 28.906527]])
cropland24 = ee.Geometry.Polygon([[112.497856, 28.842378],
   [112.511077, 28.83986],
   [112.51112, 28.833318],
   [112.505325, 28.82234],
   [112.497856, 28.842378]])
cropland25 = ee.Geometry.Polygon([[112.440925, 28.854408],
   [112.447213, 28.857904],
   [112.452042, 28.859144],
   [112.455068, 28.858167],
   [112.455218, 28.855066],
   [112.454724, 28.852961],
   [112.452085, 28.855103],
   [112.448479, 28.853919],
   [112.447878, 28.850856],
   [112.447621, 28.847078],
   [112.444015, 28.844766],
   [112.440925, 28.854408]])
cropland26 = ee.Geometry.Polygon([[112.852868, 29.026074],
   [112.844627, 29.014891],
   [112.846601, 29.001004],
   [112.869007, 29.018719],
   [112.852868, 29.026074]])
cropland27 = ee.Geometry.Polygon([[112.966296, 28.882726],
   [112.97913, 28.882238],
   [112.979946, 28.87072],
   [112.977328, 28.87243],
   [112.966425, 28.874346],
   [112.966296, 28.882726]])
cropland28 = ee.Geometry.Polygon([[112.869757, 28.743428],
   [112.869757, 28.752759],
   [112.882977, 28.752759],
   [112.882977, 28.743428],
   [112.869757, 28.743428]])
cropland29 = ee.Geometry.Polygon([[112.839594, 28.705499],
   [112.842347, 28.703142],
   [112.841628, 28.702549],
   [112.838822, 28.704981],
   [112.839594, 28.705499]])
cropland30 = ee.Geometry.Polygon([[112.688162, 28.754886],
   [112.685522, 28.74817],
   [112.688376, 28.746872],
   [112.694901, 28.752722],
   [112.688162, 28.754886]])
cropland31 = ee.Geometry.Polygon([[112.4329, 28.831327],
   [112.439767, 28.832286],
   [112.455627, 28.82471],
   [112.454833, 28.822116],
   [112.432642, 28.82721],
   [112.4329, 28.831327]])
cropland32 = ee.Geometry.Polygon([[112.712988, 29.109199],
   [112.735952, 29.113811],
   [112.738355, 29.104362],
   [112.716207, 29.100575],
   [112.712988, 29.109199]])
cropland33 = ee.Geometry.Polygon([[113.014245, 28.997415],
   [113.012614, 28.996307],
   [113.009738, 28.995735],
   [113.008805, 28.994684],
   [113.00962, 28.992366],
   [113.015114, 28.996016],
   [113.014245, 28.997415]])
cropland34 = ee.Geometry.Polygon([[112.913651, 29.509443],
   [112.915465, 29.509303],
   [112.914832, 29.504504],
   [112.912117, 29.504821],
   [112.912514, 29.508276],
   [112.913641, 29.508239],
   [112.913651, 29.509443]])

bareground1 = ee.Geometry.Polygon([[113.065046, 29.043038],
   [113.066011, 29.043113],
   [113.066161, 29.042851],
   [113.066515, 29.042832],
   [113.066676, 29.04316],
   [113.06702, 29.043141],
   [113.066902, 29.042072],
   [113.065775, 29.041069],
   [113.065667, 29.040525],
   [113.065753, 29.039418],
   [113.064798, 29.039549],
   [113.064133, 29.039483],
   [113.064197, 29.040862],
   [113.064433, 29.041472],
   [113.065249, 29.041125],
   [113.065582, 29.041613],
   [113.065249, 29.041838],
   [113.065571, 29.042007],
   [113.065667, 29.042419],
   [113.064906, 29.042513],
   [113.065046, 29.043038]])
bareground2 = ee.Geometry.Polygon([[112.296239, 28.936801],
   [112.295858, 28.93658],
   [112.296351, 28.935773],
   [112.296528, 28.935979],
   [112.296523, 28.936326],
   [112.296738, 28.936369],
   [112.296952, 28.936542],
   [112.296239, 28.936801]])
bareground3 = ee.Geometry.Polygon([[113.064466, 29.042612],
   [113.064031, 29.042335],
   [113.063376, 29.041786],
   [113.063183, 29.041416],
   [113.062775, 29.041135],
   [113.062899, 29.040909],
   [113.061665, 29.039582],
   [113.061933, 29.039282],
   [113.062073, 29.039043],
   [113.06226, 29.03923],
   [113.062636, 29.039535],
   [113.062872, 29.040023],
   [113.063173, 29.040272],
   [113.063189, 29.039357],
   [113.062743, 29.039141],
   [113.062325, 29.038916],
   [113.062271, 29.038583],
   [113.062668, 29.038536],
   [113.063173, 29.038536],
   [113.063355, 29.038691],
   [113.063414, 29.03916],
   [113.063532, 29.039512],
   [113.063537, 29.040323],
   [113.063564, 29.040844],
   [113.063215, 29.0406],
   [113.063108, 29.040787],
   [113.063698, 29.041458],
   [113.063886, 29.042021],
   [113.064299, 29.042096],
   [113.064466, 29.042612]])
bareground4 = ee.Geometry.Polygon([[113.065379, 29.024165],
   [113.066023, 29.023986],
   [113.065744, 29.023386],
   [113.065271, 29.022879],
   [113.065336, 29.022176],
   [113.065722, 29.022204],
   [113.066108, 29.022438],
   [113.066473, 29.02226],
   [113.066913, 29.021688],
   [113.066913, 29.021078],
   [113.06672, 29.020524],
   [113.066194, 29.020872],
   [113.065722, 29.021453],
   [113.065132, 29.021894],
   [113.064552, 29.022148],
   [113.06437, 29.022682],
   [113.06466, 29.023001],
   [113.064778, 29.023555],
   [113.065357, 29.023724],
   [113.065379, 29.024165]])
bareground5 = ee.Geometry.Polygon([[113.072507, 29.016059],
   [113.07313, 29.015641],
   [113.073849, 29.015566],
   [113.074396, 29.015993],
   [113.074568, 29.01612],
   [113.075012, 29.015932],
   [113.075087, 29.015163],
   [113.074926, 29.014431],
   [113.074631, 29.013812],
   [113.075522, 29.014206],
   [113.075672, 29.013957],
   [113.074937, 29.012451],
   [113.074379, 29.01164],
   [113.073896, 29.01194],
   [113.073016, 29.011048],
   [113.072544, 29.011686],
   [113.072394, 29.012409],
   [113.072587, 29.013038],
   [113.073006, 29.013328],
   [113.073199, 29.012634],
   [113.073263, 29.011977],
   [113.074341, 29.012257],
   [113.074138, 29.013028],
   [113.073188, 29.014065],
   [113.073086, 29.014731],
   [113.072528, 29.014778],
   [113.072507, 29.016059]])
bareground6 = ee.Geometry.Polygon([[113.08477, 29.027556],
   [113.086803, 29.02734],
   [113.085596, 29.025065],
   [113.084357, 29.025239],
   [113.08477, 29.027556]])
bareground7 = ee.Geometry.Polygon([[113.064546, 28.984366],
   [113.064111, 28.983662],
   [113.06548, 28.981808],
   [113.066343, 28.982324],
   [113.065485, 28.98285],
   [113.065828, 28.983329],
   [113.065673, 28.983516],
   [113.064964, 28.983254],
   [113.06482, 28.983713],
   [113.065211, 28.984098],
   [113.064546, 28.984366]])
bareground8 = ee.Geometry.Polygon([[113.066816, 28.984408],
   [113.067492, 28.98453],
   [113.068146, 28.984577],
   [113.068699, 28.984802],
   [113.068275, 28.983915],
   [113.068302, 28.983521],
   [113.068135, 28.983305],
   [113.067411, 28.983347],
   [113.066842, 28.984319],
   [113.066816, 28.984408]])
bareground9 = ee.Geometry.Polygon([[113.072861, 29.002398],
   [113.071911, 29.001417],
   [113.071675, 29.000375],
   [113.071772, 28.999667],
   [113.071101, 28.998977],
   [113.071246, 28.998278],
   [113.071611, 28.998194],
   [113.071938, 28.998428],
   [113.071611, 28.998766],
   [113.071836, 28.999132],
   [113.072013, 28.999559],
   [113.071922, 28.999977],
   [113.072051, 29.000192],
   [113.072206, 29.000108],
   [113.072426, 29.00091],
   [113.072963, 29.001065],
   [113.073049, 29.00137],
   [113.072861, 29.002398]])
bareground10 = ee.Geometry.Polygon([[112.302535, 28.988397],
   [112.300985, 28.98713],
   [112.300722, 28.98651],
   [112.300974, 28.985215],
   [112.301259, 28.985459],
   [112.301516, 28.986482],
   [112.30202, 28.98652],
   [112.302353, 28.986093],
   [112.302208, 28.985379],
   [112.302498, 28.985112],
   [112.302997, 28.984807],
   [112.30334, 28.985178],
   [112.303002, 28.985605],
   [112.303394, 28.98598],
   [112.303324, 28.986421],
   [112.303507, 28.986843],
   [112.303694, 28.987402],
   [112.303394, 28.987852],
   [112.302535, 28.988397]])
bareground11 = ee.Geometry.Polygon([[112.306605, 28.942608],
   [112.307267, 28.941749],
   [112.308394, 28.941789],
   [112.309387, 28.941836],
   [112.309647, 28.941383],
   [112.309016, 28.94097],
   [112.308295, 28.940944],
   [112.307898, 28.940613],
   [112.30709, 28.94084],
   [112.306275, 28.941463],
   [112.305827, 28.942176],
   [112.306605, 28.942608]])
bareground12 = ee.Geometry.Polygon([[112.277688, 28.945535],
   [112.278061, 28.945493],
   [112.278039, 28.94495],
   [112.277428, 28.943967],
   [112.277079, 28.944012],
   [112.277092, 28.944263],
   [112.277277, 28.944561],
   [112.277224, 28.944711],
   [112.277658, 28.944737],
   [112.277688, 28.945535]])
bareground13 = ee.Geometry.Polygon([[112.335543, 28.871141],
   [112.336393, 28.871155],
   [112.336428, 28.870873],
   [112.335696, 28.870432],
   [112.33542, 28.870491],
   [112.335543, 28.871141]])
bareground14 = ee.Geometry.Polygon([[112.340548, 28.800682],
   [112.341524, 28.800504],
   [112.341396, 28.799667],
   [112.342812, 28.799648],
   [112.343102, 28.800382],
   [112.344765, 28.800419],
   [112.344593, 28.799488],
   [112.34307, 28.798868],
   [112.339936, 28.798887],
   [112.340548, 28.800682]])
bareground15 = ee.Geometry.Polygon([[113.036871, 29.011837],
   [113.03833, 29.010626],
   [113.038352, 29.00829],
   [113.037708, 29.008421],
   [113.03788, 29.010551],
   [113.03701, 29.010607],
   [113.036871, 29.011837]])
bareground16 = ee.Geometry.Polygon([[113.063834, 29.021041],
   [113.064757, 29.021017],
   [113.064837, 29.019807],
   [113.063517, 29.020473],
   [113.063834, 29.021041]])  
bareground17 = ee.Geometry.Polygon([[113.046732, 28.966762],
   [113.047409, 28.96634],
   [113.047441, 28.964181],
   [113.046282, 28.964875],
   [113.046732, 28.966762]])
bareground18 = ee.Geometry.Polygon([[112.985169, 28.98014],
   [112.986154, 28.978866],
   [112.987133, 28.979399],
   [112.986221, 28.980616],
   [112.985169, 28.98014]])
bareground19 = ee.Geometry.Polygon([[113.011208, 28.974328],
   [113.011967, 28.973763],
   [113.010945, 28.972831],
   [113.010092, 28.973465],
   [113.011208, 28.974328]])
bareground20 = ee.Geometry.Polygon([[112.922654, 28.76159],
   [112.923878, 28.761602],
   [112.923824, 28.75981],
   [112.922598, 28.759836],
   [112.922654, 28.76159]])
bareground21 = ee.Geometry.Polygon([[112.993763, 28.744662],
   [112.994885, 28.744906],
   [112.995448, 28.744149],
   [112.995303, 28.743044],
   [112.99423, 28.743157],
   [112.994268, 28.74398],
   [112.993506, 28.744093],
   [112.993763, 28.744662]])


urban_area1 = ee.Geometry.Polygon([[113.10221, 29.140903],
   [113.10221, 29.151398],
   [113.117662, 29.151398],
   [113.117662, 29.140903],
   [113.10221, 29.140903]])
urban_area2 = ee.Geometry.Polygon([[112.354556, 28.839219],
   [112.354556, 28.847151],
   [112.366789, 28.847151],
   [112.366789, 28.839219],
   [112.354556, 28.839219]])
urban_area3 = ee.Geometry.Polygon([[113.134333, 29.367364],
   [113.139527, 29.367215],
   [113.140664, 29.364298],
   [113.141909, 29.364559],
   [113.141823, 29.366168],
   [113.144141, 29.366055],
   [113.143968, 29.362988],
   [113.140426, 29.36254],
   [113.139139, 29.364541],
   [113.137808, 29.365139],
   [113.13519, 29.365083],
   [113.134333, 29.367364]])
urban_area4 = ee.Geometry.Polygon([[113.16387, 29.363569],
   [113.17062, 29.363223],
   [113.170395, 29.360586],
   [113.163827, 29.360838],
   [113.16387, 29.363569]])
urban_area5 = ee.Geometry.Polygon([[113.1586, 29.366365],
   [113.16286, 29.366318],
   [113.162838, 29.364158],
   [113.158535, 29.364289],
   [113.1586, 29.366365]])
urban_area6 = ee.Geometry.Polygon([[113.161981, 29.371488],
   [113.163504, 29.371357],
   [113.163472, 29.369441],
   [113.162946, 29.369095],
   [113.159985, 29.370245],
   [113.159051, 29.370376],
   [113.159287, 29.371086],
   [113.161015, 29.37117],
   [113.161981, 29.371488]])
urban_area7 = ee.Geometry.Polygon([[113.159578, 29.378285],
   [113.162787, 29.37807],
   [113.162636, 29.37678],
   [113.159321, 29.376808],
   [113.159578, 29.378285]])
urban_area8 = ee.Geometry.Polygon([[113.156498, 29.38052],
   [113.158355, 29.380482],
   [113.158269, 29.379594],
   [113.157164, 29.379576],
   [113.157249, 29.379912],
   [113.156466, 29.379912],
   [113.156498, 29.38052]])
urban_area9 = ee.Geometry.Polygon([[113.179474, 29.387915],
   [113.179828, 29.387901],
   [113.179897, 29.387256],
   [113.180643, 29.386517],
   [113.1806, 29.385068],
   [113.179635, 29.385101],
   [113.179683, 29.38547],
   [113.179007, 29.385517],
   [113.179474, 29.387915]])
urban_area10 = ee.Geometry.Polygon([[113.159174, 29.393421],
   [113.160976, 29.39465],
   [113.161744, 29.393603],
   [113.161588, 29.393117],
   [113.162602, 29.392145],
   [113.161057, 29.391294],
   [113.160633, 29.390911],
   [113.158959, 29.393257],
   [113.159174, 29.393421]])
urban_area11 = ee.Geometry.Polygon([[113.134782, 29.386709],
   [113.137325, 29.38454],
   [113.137937, 29.385297],
   [113.140136, 29.38382],
   [113.138216, 29.381249],
   [113.13535, 29.383147],
   [113.136198, 29.384278],
   [113.13417, 29.385933],
   [113.134782, 29.386709]])
urban_area12 = ee.Geometry.Polygon([[113.104172, 29.375424],
   [113.107874, 29.375125],
   [113.107606, 29.373751],
   [113.104172, 29.375424]])
urban_area13 = ee.Geometry.Polygon([[113.103942, 29.374635],
   [113.102654, 29.37248],
   [113.105385, 29.372358],
   [113.106818, 29.373475],
   [113.103942, 29.374635]])
urban_area14 = ee.Geometry.Polygon([[113.102049, 29.367917],
   [113.10383, 29.367907],
   [113.103701, 29.366963],
   [113.104409, 29.365888],
   [113.103089, 29.365252],
   [113.101448, 29.366271],
   [113.101362, 29.367234],
   [113.102049, 29.367917]])
urban_area15 = ee.Geometry.Polygon([[113.190994, 29.376163],
   [113.193473, 29.375976],
   [113.192282, 29.371329],
   [113.190479, 29.371404],
   [113.190994, 29.376163]])
urban_area16 = ee.Geometry.Polygon([[113.082328, 29.288766],
   [113.086041, 29.288794],
   [113.08617, 29.286604],
   [113.082403, 29.286398],
   [113.082328, 29.288766]])
urban_area17 = ee.Geometry.Polygon([[112.357028, 28.819047],
   [112.357907, 28.818324],
   [112.356437, 28.817149],
   [112.355675, 28.817835],
   [112.357028, 28.819047]])
urban_area18 = ee.Geometry.Polygon([[112.34574, 28.818309],
   [112.347886, 28.818385],
   [112.347709, 28.818046],
   [112.347785, 28.814949],
   [112.345585, 28.814817],
   [112.345343, 28.817134],
   [112.345783, 28.817205],
   [112.34574, 28.818309]])
urban_area19 = ee.Geometry.Polygon([[112.368518, 28.828386],
   [112.371748, 28.828893],
   [112.3731, 28.828498],
   [112.373218, 28.830294],
   [112.374109, 28.83128],
   [112.374281, 28.825735],
   [112.372596, 28.824175],
   [112.371383, 28.824175],
   [112.37134, 28.825867],
   [112.370149, 28.825895],
   [112.368593, 28.827549],
   [112.368518, 28.828386]])
urban_area20 = ee.Geometry.Polygon([[113.173161, 29.335878],
   [113.174953, 29.335822],
   [113.174824, 29.332839],
   [113.173129, 29.332764],
   [113.173161, 29.335878]])
urban_area21 = ee.Geometry.Polygon([[113.170242, 29.332427],
   [113.174373, 29.332315],
   [113.174524, 29.329668],
   [113.170306, 29.330014],
   [113.170242, 29.332427]]) 
urban_area22 = ee.Geometry.Polygon([[112.685576, 29.452126],
   [112.688881, 29.452144],
   [112.688935, 29.449678],
   [112.685479, 29.449585],
   [112.685576, 29.452126]])
urban_area23 = ee.Geometry.Polygon([[112.910957, 28.850866],
   [112.914166, 28.850781],
   [112.914069, 28.848526],
   [112.911107, 28.848404],
   [112.910957, 28.850866]])
urban_area24 = ee.Geometry.Polygon([[112.914267, 28.855048],
   [112.916654, 28.855039],
   [112.916654, 28.853465],
   [112.914272, 28.853497],
   [112.914267, 28.855048]])
urban_area25 = ee.Geometry.Polygon([[112.908511, 28.856072],
   [112.911773, 28.856119],
   [112.911805, 28.852633],
   [112.908372, 28.852548],
   [112.908511, 28.856072]])
urban_area26 = ee.Geometry.Polygon([[112.930233, 28.862119],
   [112.932245, 28.858642],
   [112.930292, 28.85868],
   [112.930137, 28.859497],
   [112.929327, 28.85953],
   [112.930233, 28.862119]])
urban_area27 = ee.Geometry.Polygon([[112.897605, 28.69158],
   [112.902343, 28.691674],
   [112.90223, 28.68718],
   [112.897417, 28.687896],
   [112.897605, 28.69158]])
urban_area28 = ee.Geometry.Polygon([[112.871313, 28.706736],
   [112.875273, 28.707724],
   [112.877484, 28.701749],
   [112.872773, 28.70173],
   [112.871313, 28.706736]])
urban_area29 = ee.Geometry.Polygon([[111.942035, 28.903918],
   [111.948398, 28.904285],
   [111.948377, 28.90312],
   [111.945941, 28.903026],
   [111.945952, 28.89972],
   [111.942003, 28.899757],
   [111.942035, 28.903918]])
urban_area30 = ee.Geometry.Polygon([[111.956166, 28.909103],
   [111.959943, 28.909638],
   [111.960737, 28.90683],
   [111.957528, 28.906661],
   [111.956166, 28.909103]])
urban_area31 = ee.Geometry.Polygon([[113.023623, 29.078881],
   [113.023794, 29.073134],
   [113.025812, 29.073068],
   [113.025576, 29.078881],
   [113.023623, 29.078881]])

In [ ]:
# Make a FeatureCollection from the hand-made geometries.
wetpolygons = ee.FeatureCollection([
  ee.Feature(water_body1, {'class': 2}),
  ee.Feature(water_body2, {'class': 2}),
  ee.Feature(water_body3, {'class': 2}),
  ee.Feature(water_body4, {'class': 2}),
  ee.Feature(water_body5, {'class': 2}),
  ee.Feature(water_body6, {'class': 2}),
  ee.Feature(water_body7, {'class': 2}),
  ee.Feature(water_body8, {'class': 2}),
  ee.Feature(water_body9, {'class': 2}),
  ee.Feature(water_body10, {'class': 2}),
  ee.Feature(water_body11, {'class': 2}),
  ee.Feature(water_body12, {'class': 2}),
  ee.Feature(water_body13, {'class': 2}),
  ee.Feature(water_body14, {'class': 2}),
  ee.Feature(water_body15, {'class': 2}),
  ee.Feature(water_body16, {'class': 2}),
  ee.Feature(water_body17, {'class': 2}),
  ee.Feature(water_body18, {'class': 2}),
  ee.Feature(water_body19, {'class': 2}),
  ee.Feature(water_body20, {'class': 2}),
  ee.Feature(water_body21, {'class': 2}),
  ee.Feature(water_body22, {'class': 2}),
  ee.Feature(water_body23, {'class': 2}),
  ee.Feature(water_body24, {'class': 2}),

  ee.Feature(cropland1, {'class': 1}),
  ee.Feature(cropland2, {'class': 1}),
  ee.Feature(cropland3, {'class': 1}),
  ee.Feature(cropland4, {'class': 1}),
  ee.Feature(cropland5, {'class': 1}),
  ee.Feature(cropland6, {'class': 1}),
  ee.Feature(cropland7, {'class': 1}),
  ee.Feature(cropland8, {'class': 1}),
  ee.Feature(cropland9, {'class': 1}),
  ee.Feature(cropland10, {'class': 1}),
  ee.Feature(cropland11, {'class': 1}),
  ee.Feature(cropland12, {'class': 1}),
  ee.Feature(cropland13, {'class': 1}),
  ee.Feature(cropland14, {'class': 1}),
  ee.Feature(cropland15, {'class': 1}),
  ee.Feature(cropland16, {'class': 1}),
  ee.Feature(cropland17, {'class': 1}),
  ee.Feature(cropland18, {'class': 1}),
  ee.Feature(cropland19, {'class': 1}),
  ee.Feature(cropland20, {'class': 1}),
  ee.Feature(cropland21, {'class': 1}),
  ee.Feature(cropland22, {'class': 1}),
  ee.Feature(cropland23, {'class': 1}),
  ee.Feature(cropland24, {'class': 1}),
  ee.Feature(cropland25, {'class': 1}),
  ee.Feature(cropland26, {'class': 1}),
  ee.Feature(cropland27, {'class': 1}),
  ee.Feature(cropland28, {'class': 1}),
  ee.Feature(cropland29, {'class': 1}),
  ee.Feature(cropland30, {'class': 1}),
  ee.Feature(cropland31, {'class': 1}),
  ee.Feature(cropland32, {'class': 1}),
  ee.Feature(cropland33, {'class': 1}),
  ee.Feature(cropland34, {'class': 1}),
    
  ee.Feature(bareground1, {'class': 1}),
  ee.Feature(bareground2, {'class': 1}),
  ee.Feature(bareground3, {'class': 1}),
  ee.Feature(bareground4, {'class': 1}),
  ee.Feature(bareground5, {'class': 1}),
  ee.Feature(bareground6, {'class': 1}),
  ee.Feature(bareground7, {'class': 1}),
  ee.Feature(bareground8, {'class': 1}),
  ee.Feature(bareground9, {'class': 1}),
  ee.Feature(bareground10, {'class': 1}),
  ee.Feature(bareground11, {'class': 1}),
  ee.Feature(bareground12, {'class': 1}), 
  ee.Feature(bareground13, {'class': 1}),
  ee.Feature(bareground14, {'class': 1}),
  ee.Feature(bareground15, {'class': 1}),
  ee.Feature(bareground16, {'class': 1}),
  ee.Feature(bareground17, {'class': 1}),
  ee.Feature(bareground18, {'class': 1}),
  ee.Feature(bareground19, {'class': 1}),
  ee.Feature(bareground20, {'class': 1}),
  ee.Feature(bareground21, {'class': 1}),
    
  ee.Feature(urban_area1, {'class': 1}),
  ee.Feature(urban_area2, {'class': 1}),
  ee.Feature(urban_area3, {'class': 1}),
  ee.Feature(urban_area4, {'class': 1}),
  ee.Feature(urban_area5, {'class': 1}),
  ee.Feature(urban_area6, {'class': 1}),
  ee.Feature(urban_area7, {'class': 1}),
  ee.Feature(urban_area8, {'class': 1}),
  ee.Feature(urban_area9, {'class': 1}),
  ee.Feature(urban_area10, {'class': 1}),
  ee.Feature(urban_area11, {'class': 1}),
  ee.Feature(urban_area12, {'class': 1}),
  ee.Feature(urban_area13, {'class': 1}),
  ee.Feature(urban_area14, {'class': 1}),
  ee.Feature(urban_area15, {'class': 1}),
  ee.Feature(urban_area16, {'class': 1}),
  ee.Feature(urban_area17, {'class': 1}),
  ee.Feature(urban_area18, {'class': 1}),
  ee.Feature(urban_area19, {'class': 1}),
  ee.Feature(urban_area20, {'class': 1}),
  ee.Feature(urban_area21, {'class': 1}),
  ee.Feature(urban_area22, {'class': 1}),
  ee.Feature(urban_area23, {'class': 1}),
  ee.Feature(urban_area24, {'class': 1}),
  ee.Feature(urban_area25, {'class': 1}),
  ee.Feature(urban_area26, {'class': 1}),
  ee.Feature(urban_area27, {'class': 1}),
  ee.Feature(urban_area28, {'class': 1}),
  ee.Feature(urban_area29, {'class': 1}),
  ee.Feature(urban_area30, {'class': 1}),
  ee.Feature(urban_area31, {'class': 1}),
])

classProperty = 'class'

# Get the values for all pixels in each polygon in the training.
training_wet = image_wet.select(bands).sampleRegions(collection = wetpolygons, properties = ['class'], scale = 10, tileScale = 16)

# Create an classifier with custom parameters.
classifier_wet = ee.Classifier.smileCart().train(**{
  'features': training_wet,
  'classProperty': classProperty
})

# Classify the image.
classified_wet = image_wet.classify(classifier_wet).clip(roi)

# Accuracy assessment
# random uniforms to the training dataset.
withRandom = training_wet.randomColumn('random')

# We want to reserve some of the data for testing, to avoid overfitting the model.
split = 0.7;  # Roughly 70% training, 30% testing.
trainingPartition = withRandom.filter(ee.Filter.lt('random', split))
testingPartition = withRandom.filter(ee.Filter.gte('random', split))

# Trained with 70% of our data.
trainedClassifier = ee.Classifier.smileRandomForest(8).train(**{
  'features': trainingPartition,
  'classProperty': classProperty,
  'inputProperties': bands
})

# Classify the test FeatureCollection.
test = testingPartition.classify(trainedClassifier)

# Print the confusion matrix.
confusionMatrix = test.errorMatrix(classProperty, 'classification')

# print('Confusion Matrix', confusionMatrix.array().getInfo())
# print(confusionMatrix.array())

Train roi for dry season

In [ ]:

# Use these bands for prediction.
bands = ['B2','B3','B4','B5','B6','B7']

# Manually created trainning polygons.
water_body1 = ee.Geometry.Polygon([[112.96409, 29.132637],
   [112.96409, 29.141259],
   [112.972503, 29.141259],
   [112.972503, 29.132637],
   [112.96409, 29.132637]])
water_body2 = ee.Geometry.Polygon([[112.970958, 29.346718],
   [112.972932, 29.346083],
   [112.970014, 29.333962],
   [112.960958, 29.322999],
   [112.955208, 29.316376],
   [112.948427, 29.310127],
   [112.947826, 29.311998],
   [112.963319, 29.331193],
   [112.967954, 29.336767],
   [112.970958, 29.346718]])
water_body3 = ee.Geometry.Polygon([[112.91645, 29.028523],
   [112.898595, 29.021393],
   [112.898509, 29.009684],
   [112.881941, 29.006231],
   [112.874558, 28.991742],
   [112.882541, 28.98949],
   [112.890182, 29.001427],
   [112.897822, 28.992643],
   [112.92246, 29.007732],
   [112.927439, 29.026496],
   [112.928383, 29.035952],
   [112.91645, 29.028523]])
water_body4 = ee.Geometry.Polygon([[112.813107, 29.499719],
   [112.808042, 29.495386],
   [112.816112, 29.486421],
   [112.825039, 29.490306],
   [112.813107, 29.499719]])
water_body5 = ee.Geometry.Polygon([[112.684626, 28.845317],
   [112.670977, 28.840656],
   [112.66265, 28.834115],
   [112.654151, 28.837498],
   [112.633806, 28.839679],
   [112.633205, 28.837122],
   [112.651233, 28.832009],
   [112.667028, 28.831859],
   [112.665655, 28.827573],
   [112.667457, 28.823437],
   [112.685227, 28.829603],
   [112.677158, 28.835543],
   [112.684626, 28.845317]])
water_body6 = ee.Geometry.Polygon([[112.572452, 28.746713],
   [112.571068, 28.744991],
   [112.570735, 28.744446],
   [112.570413, 28.743232],
   [112.569909, 28.742706],
   [112.567184, 28.742047],
   [112.566143, 28.741106],
   [112.566658, 28.740843],
   [112.567388, 28.741614],
   [112.569963, 28.742245],
   [112.570853, 28.743082],
   [112.571164, 28.744399],
   [112.57287, 28.746111],
   [112.572452, 28.746713]])
water_body7 = ee.Geometry.Polygon([[112.534291, 28.808964],
   [112.534291, 28.818102],
   [112.543262, 28.818102],
   [112.543262, 28.808964],
   [112.534291, 28.808964]])
water_body8 = ee.Geometry.Polygon([[112.344133, 28.908425],
   [112.35246, 28.90835],
   [112.357265, 28.90372],
   [112.355119, 28.896807],
   [112.347179, 28.897409],
   [112.340869, 28.902255],
   [112.344133, 28.908425]])
water_body9 = ee.Geometry.Polygon([[112.327135, 28.838254],
   [112.330612, 28.840133],
   [112.339197, 28.827836],
   [112.336042, 28.826557],
   [112.327135, 28.838254]])
water_body10 = ee.Geometry.Polygon([[112.339776, 28.819985],
   [112.343392, 28.819863],
   [112.341772, 28.817851],
   [112.341804, 28.815981],
   [112.339154, 28.816169],
   [112.339862, 28.818105],
   [112.339776, 28.819985]])
water_body11 = ee.Geometry.Polygon([[112.194569, 28.81656],
   [112.205514, 28.815658],
   [112.1962, 28.804076],
   [112.194569, 28.81656]])
water_body12 = ee.Geometry.Polygon([[112.157138, 28.810622],
   [112.162021, 28.811337],
   [112.159156, 28.80909],
   [112.157138, 28.810622]])
water_body13 = ee.Geometry.Polygon([[112.118927, 28.810123],
   [112.118927, 28.811143],
   [112.122179, 28.811143],
   [112.122179, 28.810123],
   [112.118927, 28.810123]])
water_body14 = ee.Geometry.Polygon([[112.17703, 28.851301],
   [112.181687, 28.851395],
   [112.181773, 28.849609],
   [112.178822, 28.848397],
   [112.174594, 28.848641],
   [112.17703, 28.851301]])
water_body6 = ee.Geometry.Polygon([[112.190731, 28.841246],
   [112.190731, 28.842232],
   [112.19336, 28.842232],
   [112.19336, 28.841246],
   [112.190731, 28.841246]])
water_body15 = ee.Geometry.Polygon([[112.475816, 28.962027],
   [112.495303, 28.958422],
   [112.494445, 28.955624],
   [112.485882, 28.95784],
   [112.47176, 28.960506],
   [112.475816, 28.962027]])
water_body6 = ee.Geometry.Polygon([[112.441705, 28.958328],
   [112.458595, 28.957877],
   [112.473275, 28.957295],
   [112.471086, 28.955268],
   [112.471193, 28.954066],
   [112.458531, 28.955418],
   [112.451749, 28.956225],
   [112.441662, 28.956601],
   [112.441705, 28.958328]])
water_body16 = ee.Geometry.Polygon([[112.821354, 28.8366],
   [112.823414, 28.834983],
   [112.817448, 28.828968],
   [112.806502, 28.824832],
   [112.795342, 28.823704],
   [112.806502, 28.827652],
   [112.814658, 28.831449],
   [112.821354, 28.8366]])
water_body17 = ee.Geometry.Polygon([[112.76219, 28.835674],
   [112.766611, 28.838005],
   [112.76571, 28.83308],
   [112.76219, 28.835674]])
water_body18 = ee.Geometry.Polygon([[112.767963, 28.820617],
   [112.778758, 28.820053],
   [112.775432, 28.817985],
   [112.767984, 28.818342],
   [112.765323, 28.819959],
   [112.767963, 28.820617]])
water_body19 = ee.Geometry.Polygon([[112.757965, 28.803714],
   [112.759746, 28.802361],
   [112.751591, 28.796155],
   [112.74996, 28.797547],
   [112.757965, 28.803714]])
water_body20 = ee.Geometry.Polygon([[112.78498, 28.810878],
   [112.781997, 28.808678],
   [112.77749, 28.807569],
   [112.775323, 28.80708],
   [112.777297, 28.806403],
   [112.779593, 28.806309],
   [112.782469, 28.807287],
   [112.785152, 28.809374],
   [112.78498, 28.810878]])
water_body21 = ee.Geometry.Polygon([[112.78736, 28.774438],
   [112.794785, 28.776094],
   [112.797447, 28.774138],
   [112.796116, 28.769962],
   [112.792253, 28.769849],
   [112.78736, 28.774438]])
water_body6 = ee.Geometry.Polygon([[112.834999, 28.740955],
   [112.834999, 28.753748],
   [112.838733, 28.753748],
   [112.838733, 28.740955],
   [112.834999, 28.740955]])
water_body22 = ee.Geometry.Polygon([[112.813593, 28.757598],
   [112.815267, 28.756836],
   [112.811725, 28.752359],
   [112.810373, 28.753196],
   [112.813593, 28.757598]])
water_body23 = ee.Geometry.Polygon([[113.027308, 29.072833],
   [113.032458, 29.07182],
   [113.030999, 29.064093],
   [113.026664, 29.056178],
   [113.021384, 29.057979],
   [113.024861, 29.065256],
   [113.027308, 29.072833]])
water_body24 = ee.Geometry.Polygon([[113.050953, 29.06188],
   [113.063487, 29.053477],
   [113.061684, 29.048262],
   [113.048292, 29.057791],
   [113.050953, 29.06188]])                    
water_body25 = ee.Geometry.Polygon([[112.938591, 29.481306],
   [112.94636, 29.485453],
   [112.950352, 29.478467],
   [112.956361, 29.474432],
   [112.963916, 29.473909],
   [112.966792, 29.469613],
   [112.95855, 29.46696],
   [112.950309, 29.468417],
   [112.942755, 29.474768],
   [112.938591, 29.481306]])
water_body26 = ee.Geometry.Polygon([[112.072347, 28.985323],
   [112.077455, 28.985642],
   [112.083593, 28.984384],
   [112.087778, 28.982789],
   [112.086812, 28.980086],
   [112.08106, 28.981569],
   [112.076746, 28.981531],
   [112.073077, 28.980968],
   [112.072347, 28.985323]])
water_body27 = ee.Geometry.Polygon([[113.103315, 29.33839],
   [113.132502, 29.343328],
   [113.129927, 29.331955],
   [113.106148, 29.328962],
   [113.103315, 29.33839]])
water_body28 = ee.Geometry.Polygon([[112.672777, 28.783258],
   [112.672777, 28.790028],
   [112.685912, 28.790028],
   [112.685912, 28.783258],
   [112.672777, 28.783258]])
water_body29 = ee.Geometry.Polygon([[112.579586, 28.796841],
   [112.583857, 28.795469],
   [112.580337, 28.788642],
   [112.578062, 28.789206],
   [112.579586, 28.796841]])
water_body30 = ee.Geometry.Polygon([[112.572152, 28.74641],
   [112.57272, 28.745884],
   [112.571207, 28.744331],
   [112.570848, 28.744675],
   [112.572152, 28.74641]])
water_body31 = ee.Geometry.Polygon([[112.68077, 28.897897],
   [112.680641, 28.896591],
   [112.688056, 28.895783],
   [112.688308, 28.896971],
   [112.68077, 28.897897]])
water_body32 = ee.Geometry.Polygon([[111.970807, 28.995818],
   [111.972009, 28.989849],
   [111.982868, 28.990562],
   [111.987805, 28.988385],
   [111.996475, 28.975884],
   [111.998793, 28.977648],
   [111.988363, 28.994992],
   [111.970807, 28.995818]])

wetland1 = ee.Geometry.Polygon([[112.875382, 29.347338],
   [112.870145, 29.344645],
   [112.874394, 29.334057],
   [112.86581, 29.336152],
   [112.858856, 29.330129],
   [112.854135, 29.318904],
   [112.850529, 29.316135],
   [112.857697, 29.31374],
   [112.86332, 29.325714],
   [112.864823, 29.329343],
   [112.876369, 29.329306],
   [112.881949, 29.331738],
   [112.880103, 29.345281],
   [112.875382, 29.347338]])
wetland2 = ee.Geometry.Polygon([[113.064714, 29.374276],
   [113.060529, 29.370293],
   [113.056752, 29.366778],
   [113.052739, 29.359709],
   [113.051783, 29.35654],
   [113.055002, 29.355081],
   [113.059896, 29.363011],
   [113.065475, 29.371762],
   [113.064714, 29.374276]])
wetland3 = ee.Geometry.Polygon([[113.034684, 29.355258],
   [113.033783, 29.352321],
   [113.027645, 29.348263],
   [113.025627, 29.34701],
   [113.026572, 29.346673],
   [113.031014, 29.347234],
   [113.033182, 29.346392],
   [113.035778, 29.345233],
   [113.035564, 29.351966],
   [113.035328, 29.354622],
   [113.034684, 29.355258]])
wetland4 = ee.Geometry.Polygon([[113.051766, 29.309886],
   [113.044512, 29.299445],
   [113.043869, 29.286457],
   [113.047732, 29.281928],
   [113.05138, 29.284137],
   [113.054127, 29.288142],
   [113.056016, 29.303524],
   [113.051766, 29.309886]])
wetland5 = ee.Geometry.Polygon([[112.87525, 29.283671],
   [112.859712, 29.27611],
   [112.848037, 29.270569],
   [112.840654, 29.266302],
   [112.836448, 29.264355],
   [112.827777, 29.253048],
   [112.829666, 29.248854],
   [112.85602, 29.254546],
   [112.871129, 29.263381],
   [112.87525, 29.283671]])
wetland5 = ee.Geometry.Polygon([[112.845081, 28.912849],
   [112.852249, 28.913037],
   [112.851777, 28.907177],
   [112.85358, 28.897446],
   [112.856971, 28.889556],
   [112.851906, 28.887752],
   [112.845897, 28.899362],
   [112.845081, 28.912849]])
wetland6 = ee.Geometry.Polygon([[112.80796, 28.796569],
   [112.81075, 28.797641],
   [112.812424, 28.799747],
   [112.814356, 28.799785],
   [112.814077, 28.796851],
   [112.813154, 28.795798],
   [112.810149, 28.79529],
   [112.80796, 28.796569]])
wetland7 = ee.Geometry.Polygon([[112.741613, 28.793184],
   [112.745283, 28.793843],
   [112.736999, 28.78632],
   [112.73408, 28.786903],
   [112.741613, 28.793184]])
wetland8 = ee.Geometry.Polygon([[112.755464, 28.792581],
   [112.754112, 28.792562],
   [112.752835, 28.791603],
   [112.751419, 28.790503],
   [112.750453, 28.789261],
   [112.750442, 28.788406],
   [112.750614, 28.787578],
   [112.749638, 28.785914],
   [112.747169, 28.782127],
   [112.746504, 28.782804],
   [112.748907, 28.785625],
   [112.74983, 28.786941],
   [112.74966, 28.787842],
   [112.749735, 28.788678],
   [112.750776, 28.790145],
   [112.752482, 28.791781],
   [112.75275, 28.792421],
   [112.753565, 28.793164],
   [112.754392, 28.793652],
   [112.755368, 28.793126],
   [112.755464, 28.792581]])
wetland9 = ee.Geometry.Polygon([[112.778649, 28.805837],
   [112.781675, 28.806345],
   [112.783789, 28.807247],
   [112.785356, 28.808648],
   [112.786085, 28.808582],
   [112.78498, 28.806975],
   [112.784025, 28.805875],
   [112.78233, 28.805245],
   [112.779572, 28.805066],
   [112.778649, 28.805837]])
wetland10 = ee.Geometry.Polygon([[112.818887, 28.815503],
   [112.83009, 28.818229],
   [112.827493, 28.816293],
   [112.826012, 28.81492],
   [112.824939, 28.811536],
   [112.822471, 28.808434],
   [112.823608, 28.806911],
   [112.825819, 28.802248],
   [112.825132, 28.801834],
   [112.823544, 28.803602],
   [112.821741, 28.804918],
   [112.818071, 28.804015],
   [112.816633, 28.804692],
   [112.82157, 28.80849],
   [112.82393, 28.811818],
   [112.824724, 28.814526],
   [112.818887, 28.815503]])
wetland11 = ee.Geometry.Polygon([[112.799463, 28.807889],
   [112.802081, 28.807644],
   [112.803905, 28.808283],
   [112.80442, 28.810088],
   [112.805815, 28.811141],
   [112.806223, 28.810446],
   [112.805579, 28.809355],
   [112.805536, 28.807475],
   [112.804206, 28.807193],
   [112.802467, 28.806535],
   [112.798712, 28.806779],
   [112.795299, 28.807061],
   [112.793733, 28.807456],
   [112.793883, 28.808227],
   [112.797295, 28.807607],
   [112.799463, 28.807889]])
wetland12 = ee.Geometry.Polygon([[112.846456, 28.83855],
   [112.845662, 28.837329],
   [112.849718, 28.834528],
   [112.85225, 28.835148],
   [112.853259, 28.836671],
   [112.846456, 28.83855]])
wetland13 = ee.Geometry.Polygon([[112.848303, 29.239542],
   [112.828902, 29.246133],
   [112.814909, 29.236321],
   [112.807956, 29.224187],
   [112.815253, 29.212427],
   [112.829331, 29.219244],
   [112.845299, 29.22254],
   [112.857317, 29.232052],
   [112.848303, 29.239542]])
wetland14 = ee.Geometry.Polygon([[112.890355, 29.436939],
   [112.945983, 29.410923],
   [112.958344, 29.397763],
   [112.959203, 29.391481],
   [112.935509, 29.40853],
   [112.912331, 29.419297],
   [112.890355, 29.428866],
   [112.890355, 29.436939]])
wetland15 = ee.Geometry.Polygon([[113.000701, 29.508313],
   [113.004478, 29.511376],
   [113.008598, 29.513654],
   [113.010401, 29.513056],
   [113.008513, 29.509583],
   [113.003834, 29.508313],
   [113.000701, 29.508313]])
wetland16 = ee.Geometry.Polygon([[113.092909, 29.456024],
   [113.09438, 29.456052],
   [113.094798, 29.450718],
   [113.0935, 29.450437],
   [113.092909, 29.456024]])
wetland17 = ee.Geometry.Polygon([[112.658851, 28.808509],
   [112.658357, 28.804091],
   [112.656276, 28.802398],
   [112.651125, 28.800932],
   [112.648228, 28.80065],
   [112.648421, 28.799559],
   [112.653829, 28.798092],
   [112.655632, 28.79734],
   [112.655589, 28.799991],
   [112.657585, 28.80127],
   [112.658958, 28.799409],
   [112.66074, 28.800781],
   [112.660375, 28.805463],
   [112.658851, 28.808509]])
wetland18 = ee.Geometry.Polygon([[112.87124, 28.747149],
   [112.87124, 28.752924],
   [112.881799, 28.752924],
   [112.881799, 28.747149],
   [112.87124, 28.747149]])
wetland19 = ee.Geometry.Polygon([[112.710542, 28.788948],
   [112.713338, 28.790269],
   [112.713579, 28.788134],
   [112.711395, 28.787152],
   [112.710542, 28.788948]])
wetland20 = ee.Geometry.Polygon([[112.694032, 28.826409],
   [112.701618, 28.824116],
   [112.701307, 28.822838],
   [112.695072, 28.824595],
   [112.694032, 28.826409]])
wetland21 = ee.Geometry.Polygon([[112.525996, 28.785826],
   [112.528115, 28.787373],
   [112.531018, 28.787951],
   [112.532391, 28.788308],
   [112.532734, 28.787556],
   [112.529167, 28.786338],
   [112.526398, 28.783743],
   [112.525996, 28.785826]])

dry_cropland1 = ee.Geometry.Polygon([[112.937211, 29.446489],
   [112.937994, 29.446134],
   [112.937297, 29.444882],
   [112.93653, 29.445116],
   [112.937211, 29.446489]])
dry_cropland2 = ee.Geometry.Polygon([[112.989785, 29.471064],
   [112.992532, 29.470998],
   [112.992167, 29.467804],
   [112.989484, 29.467878],
   [112.989785, 29.471064]])
dry_cropland3 = ee.Geometry.Polygon([[112.994192, 29.470905],
   [112.995212, 29.470858],
   [112.9949, 29.467664],
   [112.993849, 29.467701],
   [112.994192, 29.470905]])
dry_cropland4 = ee.Geometry.Polygon([[112.982851, 29.467738],
   [112.984718, 29.46772],
   [112.984493, 29.4663],
   [112.982765, 29.4663],
   [112.982851, 29.467738]])
dry_cropland5 = ee.Geometry.Polygon([[112.976649, 29.462675],
   [112.978366, 29.462535],
   [112.977894, 29.46006],
   [112.976317, 29.460209],
   [112.976649, 29.462675]])
dry_cropland6 = ee.Geometry.Polygon([[112.975137, 29.452848],
   [112.980781, 29.452596],
   [112.980738, 29.452138],
   [112.97929, 29.450708],
   [112.978625, 29.449924],
   [112.978506, 29.448924],
   [112.974643, 29.449083],
   [112.975137, 29.452848]])
dry_cropland7 = ee.Geometry.Polygon([[112.971082, 29.45018],
   [112.974054, 29.450072],
   [112.973979, 29.449217],
   [112.970969, 29.449297],
   [112.971082, 29.45018]])
dry_cropland8 = ee.Geometry.Polygon([[112.967308, 29.450413],
   [112.968936, 29.450392],
   [112.968882, 29.449439],
   [112.967262, 29.449464],
   [112.967308, 29.450413]])
dry_cropland9 = ee.Geometry.Polygon([[113.03495, 29.453408],
   [113.039296, 29.453221],
   [113.039275, 29.451232],
   [113.035122, 29.451176],
   [113.03495, 29.453408]])
dry_cropland10 = ee.Geometry.Polygon([[113.057738, 29.447607],
   [113.063522, 29.448728],
   [113.063211, 29.446448],
   [113.058135, 29.445271],
   [113.057738, 29.447607]])
dry_cropland11 = ee.Geometry.Polygon([[112.447665, 28.942451],
   [112.451475, 28.942357],
   [112.45156, 28.939427],
   [112.447611, 28.939399],
   [112.447665, 28.942451]])
dry_cropland12 = ee.Geometry.Polygon([[112.433181, 28.943568],
   [112.43184, 28.941211],
   [112.434662, 28.939916],
   [112.436078, 28.942291],
   [112.433181, 28.943568]])
dry_cropland13 = ee.Geometry.Polygon([[112.419426, 28.928004],
   [112.420891, 28.928525],
   [112.422442, 28.92567],
   [112.421733, 28.925304],
   [112.421197, 28.926144],
   [112.420548, 28.925882],
   [112.420344, 28.926332],
   [112.42088, 28.926689],
   [112.420355, 28.927746],
   [112.419705, 28.927534],
   [112.419426, 28.928004]])
dry_cropland14 = ee.Geometry.Polygon([[112.433943, 28.927478],
   [112.434732, 28.926943],
   [112.434421, 28.926586],
   [112.435199, 28.926013],
   [112.433648, 28.924027],
   [112.432312, 28.924835],
   [112.43382, 28.926614],
   [112.433519, 28.926741],
   [112.433943, 28.927478]])
dry_cropland15 = ee.Geometry.Polygon([[112.415875, 28.916768],
   [112.416074, 28.916425],
   [112.415285, 28.916049],
   [112.416025, 28.914519],
   [112.416809, 28.914828],
   [112.416535, 28.915331],
   [112.41954, 28.916505],
   [112.419116, 28.917209],
   [112.418236, 28.916777],
   [112.417796, 28.917585],
   [112.415875, 28.916768]])
dry_cropland16 = ee.Geometry.Polygon([[112.41934, 28.91166],
   [112.427538, 28.913313],
   [112.428171, 28.911096],
   [112.419662, 28.909838],
   [112.41934, 28.91166]])
dry_cropland17 = ee.Geometry.Polygon([[112.39755, 28.922909],
   [112.399181, 28.922848],
   [112.39925, 28.91982],
   [112.397582, 28.919637],
   [112.39755, 28.922909]])
dry_cropland18 = ee.Geometry.Polygon([[112.469429, 28.900501],
   [112.467615, 28.898265],
   [112.468028, 28.897843],
   [112.468855, 28.89781],
   [112.469429, 28.900501]])
dry_cropland19 = ee.Geometry.Polygon([[112.500397, 28.927061],
   [112.504464, 28.927117],
   [112.504635, 28.924554],
   [112.50045, 28.924657],
   [112.500397, 28.927061]])
dry_cropland20 = ee.Geometry.Polygon([[112.511792, 28.985546],
   [112.515494, 28.986381],
   [112.516202, 28.984467],
   [112.515483, 28.984382],
   [112.515172, 28.985537],
   [112.513155, 28.98487],
   [112.513552, 28.983828],
   [112.512318, 28.983622],
   [112.511792, 28.985546]])
dry_cropland21 = ee.Geometry.Polygon([[112.961363, 29.166894],
   [112.96645, 29.165582],
   [112.963681, 29.155782],
   [112.958015, 29.158143],
   [112.961363, 29.166894]])
dry_cropland22 = ee.Geometry.Polygon([[112.725488, 29.264876],
   [112.726818, 29.264942],
   [112.727033, 29.262845],
   [112.724812, 29.262705],
   [112.724565, 29.264408],
   [112.72567, 29.26453],
   [112.725488, 29.264876]])
dry_cropland23 = ee.Geometry.Polygon([[112.714373, 29.272691],
   [112.714941, 29.272804],
   [112.715349, 29.269528],
   [112.714587, 29.269472],
   [112.714373, 29.272691]])
dry_cropland24 = ee.Geometry.Polygon([[112.774562, 29.259813],
   [112.776601, 29.259841],
   [112.776601, 29.256471],
   [112.774573, 29.256424],
   [112.774562, 29.259813]])
dry_cropland25 = ee.Geometry.Polygon([[112.039207, 29.059571],
   [112.040699, 29.059712],
   [112.040677, 29.061109],
   [112.041997, 29.061222],
   [112.042416, 29.056945],
   [112.040173, 29.056823],
   [112.039207, 29.059571]])
dry_cropland26 = ee.Geometry.Polygon([[112.044218, 29.061662],
   [112.045367, 29.061634],
   [112.045506, 29.056926],
   [112.044283, 29.05687],
   [112.044218, 29.061662]])
dry_cropland27 = ee.Geometry.Polygon([[112.047717, 29.063819],
   [112.049476, 29.063801],
   [112.049863, 29.056955],
   [112.048006, 29.05688],
   [112.047717, 29.063819]])
dry_cropland28 = ee.Geometry.Polygon([[112.057051, 29.064073],
   [112.058413, 29.064119],
   [112.058714, 29.059046],
   [112.057265, 29.059065],
   [112.057051, 29.064073]])
dry_cropland29 = ee.Geometry.Polygon([[112.099139, 29.08107],
   [112.100422, 29.080109],
   [112.09841, 29.078037],
   [112.097197, 29.07904],
   [112.099139, 29.08107]])
dry_cropland30 = ee.Geometry.Polygon([[112.094637, 29.067239],
   [112.095818, 29.067924],
   [112.098168, 29.06519],
   [112.097717, 29.064937],
   [112.095936, 29.066972],
   [112.095287, 29.066606],
   [112.094637, 29.067239]])
dry_cropland31 = ee.Geometry.Polygon([[112.105287, 29.07475],
   [112.107497, 29.073156],
   [112.105592, 29.070985],
   [112.103393, 29.072626],
   [112.105287, 29.07475]])
dry_cropland32 = ee.Geometry.Polygon([[112.038853, 28.988086],
   [112.040152, 28.988142],
   [112.040307, 28.985716],
   [112.039116, 28.985688],
   [112.038853, 28.988086]])
dry_cropland33 = ee.Geometry.Polygon([[112.178121, 29.009035],
   [112.182214, 29.008599],
   [112.182, 29.00691],
   [112.180315, 29.007065],
   [112.180294, 29.006267],
   [112.17775, 29.006375],
   [112.178121, 29.009035]])
dry_cropland34 = ee.Geometry.Polygon([[112.170008, 28.974848],
   [112.174102, 28.97428],
   [112.173479, 28.970821],
   [112.169546, 28.971182],
   [112.170008, 28.974848]])
dry_cropland35 = ee.Geometry.Polygon([[112.396444, 28.992213],
   [112.398633, 28.994118],
   [112.403881, 28.989267],
   [112.401445, 28.987596],
   [112.396444, 28.992213]])
                                    
green_veg1 = ee.Geometry.Polygon([[112.502674, 28.945838],
   [112.504083, 28.945817],
   [112.504093, 28.944632],
   [112.502642, 28.944618],
   [112.502674, 28.945838]])
green_veg2 = ee.Geometry.Polygon([[112.501094, 28.942263],
   [112.50264, 28.942263],
   [112.502704, 28.939859],
   [112.501835, 28.939775],
   [112.501824, 28.93754],
   [112.501116, 28.937578],
   [112.501094, 28.942263]])
green_veg3 = ee.Geometry.Polygon([[112.503911, 28.948854],
   [112.504574, 28.948829],
   [112.504571, 28.947693],
   [112.50297, 28.947636],
   [112.502911, 28.947986],
   [112.503857, 28.948016],
   [112.503911, 28.948854]])
green_veg4 = ee.Geometry.Polygon([[112.521652, 28.981983],
   [112.523766, 28.982528],
   [112.524351, 28.980871],
   [112.522183, 28.980317],
   [112.521652, 28.981983]])
green_veg5 = ee.Geometry.Polygon([[112.511394, 28.968633],
   [112.519452, 28.971477],
   [112.519753, 28.970538],
   [112.511748, 28.96775],
   [112.511394, 28.968633]])
green_veg6 = ee.Geometry.Polygon([[112.501346, 28.959606],
   [112.503498, 28.960184],
   [112.503782, 28.959536],
   [112.502215, 28.959414],
   [112.501679, 28.959052],
   [112.501346, 28.959606]])
green_veg7 = ee.Geometry.Polygon([[112.678726, 28.92944],
   [112.682342, 28.929318],
   [112.682364, 28.928487],
   [112.678635, 28.928459],
   [112.678726, 28.92944]])
green_veg8 = ee.Geometry.Polygon([[112.669059, 28.905822],
   [112.669902, 28.90578],
   [112.669789, 28.90476],
   [112.668893, 28.904784],
   [112.669059, 28.905822]])
green_veg9 = ee.Geometry.Polygon([[112.676033, 28.902374],
   [112.674617, 28.900989],
   [112.674794, 28.900944],
   [112.67613, 28.902309],
   [112.676033, 28.902374]])
green_veg10 = ee.Geometry.Polygon([[112.727741, 28.927981],
   [112.728717, 28.928892],
   [112.730348, 28.927484],
   [112.729554, 28.926563],
   [112.727741, 28.927981]])
green_veg11 = ee.Geometry.Polygon([[112.671779, 28.952251],
   [112.673346, 28.952612],
   [112.673566, 28.951908],
   [112.671983, 28.95157],
   [112.671779, 28.952251]])
green_veg12 = ee.Geometry.Polygon([[112.671511, 28.953814],
   [112.672876, 28.954159],
   [112.673094, 28.953351],
   [112.67172, 28.953025],
   [112.671511, 28.953814]])
green_veg13 = ee.Geometry.Polygon([[112.709894, 28.970734],
   [112.713065, 28.969772],
   [112.712437, 28.968336],
   [112.709465, 28.969096],
   [112.709894, 28.970734]])
green_veg14 = ee.Geometry.Polygon([[112.807446, 29.111496],
   [112.822555, 29.109884],
   [112.821353, 29.096985],
   [112.804227, 29.09961],
   [112.807446, 29.111496]])
green_veg15 = ee.Geometry.Polygon([[112.772813, 29.10051],
   [112.779252, 29.098898],
   [112.778865, 29.096235],
   [112.770195, 29.095223],
   [112.772813, 29.10051]])
green_veg16 = ee.Geometry.Polygon([[112.837336, 29.044675],
   [112.84019, 29.042705],
   [112.837314, 29.037546],
   [112.835769, 29.03901],
   [112.837336, 29.044675]])
green_veg17 = ee.Geometry.Polygon([[112.838087, 29.033907],
   [112.841929, 29.036177],
   [112.844826, 29.033288],
   [112.843002, 29.031299],
   [112.841542, 29.033006],
   [112.839375, 29.031693],
   [112.838087, 29.033907]])
green_veg18 = ee.Geometry.Polygon([[112.882423, 29.082622],
   [112.885943, 29.086297],
   [112.886071, 29.078909],
   [112.883925, 29.078309],
   [112.882423, 29.082622]])
green_veg19 = ee.Geometry.Polygon([[112.89268, 29.152308],
   [112.894166, 29.152509],
   [112.894011, 29.152064],
   [112.893125, 29.151816],
   [112.893115, 29.15146],
   [112.893635, 29.151563],
   [112.893302, 29.150874],
   [112.892063, 29.151497],
   [112.89268, 29.152308]])
green_veg20 = ee.Geometry.Polygon([[112.766461, 29.276897],
   [112.767234, 29.276897],
   [112.767411, 29.274515],
   [112.766509, 29.274422],
   [112.766461, 29.276897]]) 
green_veg21 = ee.Geometry.Polygon([[112.759306, 29.298443],
   [112.761882, 29.298509],
   [112.761957, 29.295861],
   [112.759693, 29.295618],
   [112.759306, 29.298443]])
green_veg22 = ee.Geometry.Polygon([[113.045905, 29.098425],
   [113.048394, 29.100262],
   [113.047944, 29.098162],
   [113.049274, 29.096325],
   [113.046849, 29.096381],
   [113.045905, 29.098425]])
green_veg23 = ee.Geometry.Polygon([[113.06202, 29.101154],
   [113.062465, 29.100892],
   [113.062229, 29.09988],
   [113.061263, 29.099983],
   [113.06202, 29.101154]])
green_veg24 = ee.Geometry.Polygon([[113.082318, 29.103438],
   [113.084228, 29.10341],
   [113.084883, 29.101995],
   [113.08322, 29.101732],
   [113.082318, 29.103438]])                                
green_veg25 = ee.Geometry.Polygon([[113.102986, 29.127651],
   [113.103589, 29.127568],
   [113.102806, 29.125333],
   [113.102269, 29.125431],
   [113.102986, 29.127651]])   
green_veg26 = ee.Geometry.Polygon([[112.292171, 28.991547],
   [112.295863, 28.990918],
   [112.295659, 28.989483],
   [112.293094, 28.989708],
   [112.291034, 28.988797],
   [112.290937, 28.990383],
   [112.292171, 28.991547]])
green_veg27 = ee.Geometry.Polygon([[112.303067, 29.001636],
   [112.304445, 29.001763],
   [112.304928, 28.999755],
   [112.302922, 29.000163],
   [112.303067, 29.001636]])
green_veg28 = ee.Geometry.Polygon([[112.293963, 29.018746],
   [112.29561, 29.018849],
   [112.29561, 29.016705],
   [112.296624, 29.01504],
   [112.294725, 29.014927],
   [112.2937, 29.017371],
   [112.293963, 29.018746]])
green_veg29 = ee.Geometry.Polygon([[112.288245, 29.030721],
   [112.289297, 29.029924],
   [112.288776, 29.02847],
   [112.287693, 29.029267],
   [112.288406, 29.030116],
   [112.288245, 29.030721]])
green_veg30 = ee.Geometry.Polygon([[112.286662, 29.05569],
   [112.287569, 29.056656],
   [112.290107, 29.05471],
   [112.290021, 29.053922],
   [112.287945, 29.054532],
   [112.286662, 29.05569]])
green_veg31 = ee.Geometry.Polygon([[112.751828, 28.780436],
   [112.746216, 28.773026],
   [112.749006, 28.770844],
   [112.754081, 28.779327],
   [112.751828, 28.780436]])
green_veg32 = ee.Geometry.Polygon([[112.844183, 28.930659],
   [112.831757, 28.923335],
   [112.833173, 28.921475],
   [112.843904, 28.923973],
   [112.844183, 28.930659]])
green_veg33 = ee.Geometry.Polygon([[112.924552, 28.958161],
   [112.928791, 28.958124],
   [112.929005, 28.953317],
   [112.924498, 28.953252],
   [112.924552, 28.958161]])
green_veg34 = ee.Geometry.Polygon([[112.930144, 28.948085],
   [112.933728, 28.948179],
   [112.933577, 28.941907],
   [112.930315, 28.941889],
   [112.930144, 28.948085]])
green_veg35 = ee.Geometry.Polygon([[112.840575, 29.166509],
   [112.857229, 29.16917],
   [112.858581, 29.164635],
   [112.841391, 29.162162],
   [112.840575, 29.166509]])
    

bareground1 = ee.Geometry.Polygon([[113.055727, 29.137854],
   [113.056532, 29.137469],
   [113.057369, 29.137432],
   [113.057449, 29.13709],
   [113.056489, 29.136635],
   [113.055679, 29.136715],
   [113.055416, 29.13716],
   [113.055727, 29.137854]])
bareground2 = ee.Geometry.Polygon([[113.023798, 29.127207],
   [113.024211, 29.127259],
   [113.024039, 29.126509],
   [113.022875, 29.125216],
   [113.022548, 29.125488],
   [113.023733, 29.126575],
   [113.023798, 29.127207]])
bareground3 = ee.Geometry.Polygon([[113.051558, 29.140885],
   [113.051988, 29.140974],
   [113.052862, 29.139775],
   [113.052707, 29.139554],
   [113.05173, 29.140215],
   [113.051558, 29.140885]])
bareground4 = ee.Geometry.Polygon([[113.044155, 29.097148],
   [113.045411, 29.096604],
   [113.044606, 29.095714],
   [113.044155, 29.097148]])
bareground5 = ee.Geometry.Polygon([[113.110428, 29.13041],
   [113.110721, 29.130417],
   [113.110713, 29.130147],
   [113.110359, 29.130129],
   [113.110428, 29.13041]])
bareground6 = ee.Geometry.Polygon([[113.090869, 29.092037],
   [113.090864, 29.091325],
   [113.091245, 29.091498],
   [113.090869, 29.092037]])
bareground7 = ee.Geometry.Polygon([[113.102371, 29.177674],
   [113.103991, 29.176887],
   [113.103734, 29.176344],
   [113.102425, 29.177009],
   [113.102371, 29.177674]])
bareground8 = ee.Geometry.Polygon([[112.894381, 28.873419],
   [112.896399, 28.873588],
   [112.896495, 28.872038],
   [112.894413, 28.872198],
   [112.894381, 28.873419]])
bareground9 = ee.Geometry.Polygon([[112.922754, 28.760122],
   [112.922754, 28.761044],
   [112.923789, 28.761044],
   [112.923789, 28.760122],
   [112.922754, 28.760122]])
bareground10 = ee.Geometry.Polygon([[112.904649, 28.774987],
   [112.904649, 28.775598],
   [112.905271, 28.775598],
   [112.905271, 28.774987],
   [112.904649, 28.774987]])
bareground11 = ee.Geometry.Polygon([[113.193684, 29.383745],
   [113.193684, 29.383977],
   [113.194499, 29.383977],
   [113.194499, 29.383745],
   [113.193684, 29.383745]])
bareground12 = ee.Geometry.Polygon([[113.200829, 29.411748],
   [113.200856, 29.411549],
   [113.20316, 29.411757],
   [113.203249, 29.411981],
   [113.200829, 29.411748]])
bareground13 = ee.Geometry.Polygon([[113.179468, 29.414832],
   [113.179468, 29.415301],
   [113.179994, 29.415301],
   [113.179994, 29.414832],
   [113.179468, 29.414832]])
bareground14 = ee.Geometry.Polygon([[113.092774, 29.245005],
   [113.095333, 29.245169],
   [113.094625, 29.243742],
   [113.094646, 29.242328],
   [113.093117, 29.242314],
   [113.093686, 29.243297],
   [113.092774, 29.245005]])
bareground15 = ee.Geometry.Polygon([[113.147979, 29.234364],
   [113.151434, 29.23413],
   [113.151638, 29.233278],
   [113.150082, 29.232557],
   [113.151262, 29.230479],
   [113.150361, 29.229477],
   [113.147979, 29.234364]])
bareground16 = ee.Geometry.Polygon([[113.150446, 29.245261],
   [113.152195, 29.243174],
   [113.153773, 29.242893],
   [113.154417, 29.241667],
   [113.155608, 29.241432],
   [113.156294, 29.241086],
   [113.153644, 29.239204],
   [113.152507, 29.240684],
   [113.153365, 29.241629],
   [113.152303, 29.242659],
   [113.151047, 29.242359],
   [113.15124, 29.243623],
   [113.150446, 29.245261]])
bareground17 = ee.Geometry.Polygon([[113.152206, 29.276791],
   [113.151068, 29.275963],
   [113.151347, 29.275411],
   [113.152007, 29.274798],
   [113.152726, 29.275626],
   [113.152206, 29.276791]])
bareground18 = ee.Geometry.Polygon([[113.188998, 29.327161],
   [113.186141, 29.326635],
   [113.185744, 29.325073],
   [113.188169, 29.325157],
   [113.188998, 29.327161]])
bareground19 = ee.Geometry.Polygon([[112.302084, 28.988497],
   [112.303565, 28.988347],
   [112.302985, 28.984696],
   [112.302266, 28.984997],
   [112.302256, 28.98677],
   [112.300936, 28.986639],
   [112.302084, 28.988497]])
bareground20 = ee.Geometry.Polygon([[112.977423, 28.713913],
   [112.978882, 28.713594],
   [112.979118, 28.710846],
   [112.977219, 28.710667],
   [112.977144, 28.71205],
   [112.976425, 28.71221],
   [112.977423, 28.713913]])
                                    
urban_area1 = ee.Geometry.Polygon([[113.137876, 29.385102],
   [113.139937, 29.383868],
   [113.137919, 29.380727],
   [113.136567, 29.381475],
   [113.13558, 29.383139],
   [113.136438, 29.384485],
   [113.135494, 29.385121],
   [113.134357, 29.386093],
   [113.134507, 29.386673],
   [113.137061, 29.385177],
   [113.137876, 29.385102]])
urban_area2 = ee.Geometry.Polygon([[113.171742, 29.36977],
   [113.175133, 29.369601],
   [113.174833, 29.362084],
   [113.171506, 29.362327],
   [113.171742, 29.36977]])
urban_area3 = ee.Geometry.Polygon([[113.145168, 29.36738],
   [113.148505, 29.367109],
   [113.148323, 29.366473],
   [113.147089, 29.366118],
   [113.146906, 29.365342],
   [113.147786, 29.364407],
   [113.146541, 29.363107],
   [113.144953, 29.362976],
   [113.145168, 29.36738]])
urban_area4 = ee.Geometry.Polygon([[113.173215, 29.335828],
   [113.174792, 29.335856],
   [113.174674, 29.332751],
   [113.172979, 29.332732],
   [113.173215, 29.335828]])
urban_area5 = ee.Geometry.Polygon([[113.110975, 29.141044],
   [113.115203, 29.141466],
   [113.115235, 29.135984],
   [113.110771, 29.137942],
   [113.110975, 29.141044]])
urban_area6 = ee.Geometry.Polygon([[113.121758, 29.135947],
   [113.122231, 29.135942],
   [113.122123, 29.134232],
   [113.121732, 29.134255],
   [113.121758, 29.135947]])                                  
urban_area7 = ee.Geometry.Polygon([[113.110621, 29.142619],
   [113.112, 29.142867],
   [113.112145, 29.142145],
   [113.110782, 29.142],
   [113.110621, 29.142619]])
urban_area8 = ee.Geometry.Polygon([[113.104232, 29.137797],
   [113.105434, 29.137226],
   [113.105133, 29.136518],
   [113.103728, 29.136879],
   [113.104232, 29.137797]])
urban_area9 = ee.Geometry.Polygon([[113.101008, 29.138926],
   [113.102435, 29.138549],
   [113.101917, 29.137467],
   [113.100734, 29.137752],
   [113.101008, 29.138926]])
urban_area10 = ee.Geometry.Polygon([[113.09368, 29.140032],
   [113.094458, 29.140046],
   [113.09456, 29.138729],
   [113.093828, 29.138645],
   [113.09368, 29.140032]])
urban_area11 = ee.Geometry.Polygon([[112.345536, 28.818346],
   [112.347521, 28.818402],
   [112.347446, 28.817678],
   [112.345418, 28.817528],
   [112.345536, 28.818346]])
urban_area4 = ee.Geometry.Polygon([[112.353364, 28.813562],
   [112.353959, 28.813567],
   [112.353804, 28.811024],
   [112.353192, 28.811047],
   [112.353203, 28.811879],
   [112.353444, 28.81195],
   [112.353364, 28.813562]])
urban_area12 = ee.Geometry.Polygon([[112.354034, 28.820974],
   [112.354909, 28.820847],
   [112.354753, 28.819038],
   [112.353954, 28.819038],
   [112.354034, 28.820974]])
urban_area13 = ee.Geometry.Polygon([[112.354732, 28.822792],
   [112.355859, 28.823666],
   [112.357103, 28.822463],
   [112.355998, 28.821777],
   [112.354732, 28.822792]])
urban_area14 = ee.Geometry.Polygon([[112.356432, 28.820518],
   [112.356531, 28.820615],
   [112.357492, 28.819752],
   [112.357411, 28.819597],
   [112.356767, 28.820093],
   [112.35636, 28.819719],
   [112.356233, 28.819818],
   [112.356676, 28.820236],
   [112.356432, 28.820518]])
urban_area15 = ee.Geometry.Polygon([[112.358391, 28.815601],
   [112.359303, 28.81561],
   [112.359185, 28.814116],
   [112.358241, 28.814135],
   [112.358391, 28.815601]])
urban_area16 = ee.Geometry.Polygon([[112.346084, 28.816334],
   [112.347769, 28.816466],
   [112.347747, 28.815742],
   [112.346105, 28.815686],
   [112.346084, 28.816334]])
urban_area17 = ee.Geometry.Polygon([[112.364425, 28.839894],
   [112.365718, 28.839791],
   [112.365697, 28.839062],
   [112.367161, 28.837413],
   [112.367725, 28.836835],
   [112.364441, 28.837051],
   [112.364425, 28.839894]])
urban_area18 = ee.Geometry.Polygon([[112.347628, 28.84642],
   [112.349892, 28.846317],
   [112.349495, 28.842088],
   [112.347371, 28.842172],
   [112.347628, 28.84642]])
urban_area19 = ee.Geometry.Polygon([[112.347591, 28.836929],
   [112.348439, 28.836948],
   [112.348579, 28.835923],
   [112.347516, 28.835905],
   [112.347591, 28.836929]])
urban_area20 = ee.Geometry.Polygon([[113.139299, 29.364942],
   [113.143549, 29.364896],
   [113.143656, 29.363727],
   [113.139342, 29.363783],
   [113.139299, 29.364942]])
urban_area21 = ee.Geometry.Polygon([[113.145077, 29.366131],
   [113.146767, 29.366117],
   [113.147733, 29.364658],
   [113.147153, 29.363835],
   [113.14497, 29.363943],
   [113.145077, 29.366131]])
urban_area22 = ee.Geometry.Polygon([[113.160049, 29.36621],
   [113.162898, 29.366084],
   [113.162828, 29.364424],
   [113.159963, 29.364331],
   [113.160049, 29.36621]])
urban_area23 = ee.Geometry.Polygon([[113.166709, 29.375871],
   [113.167387, 29.375841],
   [113.167242, 29.37434],
   [113.166676, 29.374354],
   [113.166709, 29.375871]])
urban_area24 = ee.Geometry.Polygon([[113.153682, 29.396145],
   [113.15535, 29.396514],
   [113.155619, 29.3955],
   [113.15387, 29.395238],
   [113.153682, 29.396145]])
urban_area25 = ee.Geometry.Polygon([[113.171786, 29.36383],
   [113.171786, 29.368103],
   [113.173095, 29.368103],
   [113.173095, 29.36383],
   [113.171786, 29.36383]])
urban_area26 = ee.Geometry.Polygon([[113.195198, 29.3713],
   [113.195198, 29.372591],
   [113.196604, 29.372591],
   [113.196604, 29.3713],
   [113.195198, 29.3713]])                                  
urban_area27 = ee.Geometry.Polygon([[113.195143, 29.366401],
   [113.195143, 29.368065],
   [113.197225, 29.368065],
   [113.197225, 29.366401],
   [113.195143, 29.366401]])
urban_area28 = ee.Geometry.Polygon([[113.198329, 29.412469],
   [113.198361, 29.412063],
   [113.203308, 29.412637],
   [113.203244, 29.413039],
   [113.198329, 29.412469]])
urban_area28 = ee.Geometry.Polygon([[111.953236, 28.910838],
   [111.959148, 28.911326],
   [111.96049, 28.906762],
   [111.958043, 28.905879],
   [111.955929, 28.908105],
   [111.953837, 28.907757],
   [111.953236, 28.910838]])
urban_area29 = ee.Geometry.Polygon([[111.96609, 28.907738],
   [111.970468, 28.908546],
   [111.970994, 28.906799],
   [111.96726, 28.905682],
   [111.96799, 28.902432],
   [111.965049, 28.902526],
   [111.964406, 28.905982],
   [111.96609, 28.907738]])
urban_area30 = ee.Geometry.Polygon([[111.941993, 28.895284],
   [111.94973, 28.895096],
   [111.949676, 28.892729],
   [111.942089, 28.89271],
   [111.941993, 28.895284]])
urban_area31 = ee.Geometry.Polygon([[111.948549, 28.929375],
   [111.953893, 28.929319],
   [111.953893, 28.92715],
   [111.948678, 28.927347],
   [111.948549, 28.929375]])
urban_area32 = ee.Geometry.Polygon([[112.382664, 28.845462],
   [112.381216, 28.844108],
   [112.382439, 28.842901],
   [112.384397, 28.84447],
   [112.382664, 28.845462]])
urban_area33 = ee.Geometry.Polygon([[113.08733, 29.28954],
   [113.091654, 29.289428],
   [113.091633, 29.287238],
   [113.087158, 29.287481],
   [113.08733, 29.28954]])
urban_area34 = ee.Geometry.Polygon([[112.865938, 28.705075],
   [112.871175, 28.700633],
   [112.866797, 28.698525],
   [112.865938, 28.705075]])

In [ ]:
# Make a FeatureCollection from the hand-made geometries.
drypolygons = ee.FeatureCollection([
  ee.Feature(water_body1, {'class': 2}),
  ee.Feature(water_body2, {'class': 2}),
  ee.Feature(water_body3, {'class': 2}),
  ee.Feature(water_body4, {'class': 2}),
  ee.Feature(water_body5, {'class': 2}),
  ee.Feature(water_body6, {'class': 2}),
  ee.Feature(water_body7, {'class': 2}),
  ee.Feature(water_body8, {'class': 2}),
  ee.Feature(water_body9, {'class': 2}),
  ee.Feature(water_body10, {'class': 2}),
  ee.Feature(water_body11, {'class': 2}),
  ee.Feature(water_body12, {'class': 2}),
  ee.Feature(water_body13, {'class': 2}),
  ee.Feature(water_body14, {'class': 2}),
  ee.Feature(water_body15, {'class': 2}),
  ee.Feature(water_body16, {'class': 2}),
  ee.Feature(water_body17, {'class': 2}),
  ee.Feature(water_body18, {'class': 2}),
  ee.Feature(water_body19, {'class': 2}),
  ee.Feature(water_body20, {'class': 2}),
  ee.Feature(water_body21, {'class': 2}),
  ee.Feature(water_body22, {'class': 2}),
  ee.Feature(water_body23, {'class': 2}),
  ee.Feature(water_body24, {'class': 2}),
  ee.Feature(water_body25, {'class': 2}),
  ee.Feature(water_body26, {'class': 2}),
  ee.Feature(water_body27, {'class': 2}),
  ee.Feature(water_body28, {'class': 2}),
  ee.Feature(water_body29, {'class': 2}),
  ee.Feature(water_body30, {'class': 2}),
  ee.Feature(water_body31, {'class': 2}),
  ee.Feature(water_body32, {'class': 2}),
    
 
  ee.Feature(dry_cropland1, {'class': 1}),
  ee.Feature(dry_cropland2, {'class': 1}),
  ee.Feature(dry_cropland3, {'class': 1}),
  ee.Feature(dry_cropland4, {'class': 1}),
  ee.Feature(dry_cropland5, {'class': 1}),
  ee.Feature(dry_cropland6, {'class': 1}),
  ee.Feature(dry_cropland7, {'class': 1}),
  ee.Feature(dry_cropland8, {'class': 1}),
  ee.Feature(dry_cropland9, {'class': 1}),
  ee.Feature(dry_cropland10, {'class': 1}),
  ee.Feature(dry_cropland11, {'class': 1}),
  ee.Feature(dry_cropland12, {'class': 1}),
  ee.Feature(dry_cropland13, {'class': 1}),
  ee.Feature(dry_cropland14, {'class': 1}),
  ee.Feature(dry_cropland15, {'class': 1}),
  ee.Feature(dry_cropland16, {'class': 1}),
  ee.Feature(dry_cropland17, {'class': 1}),
  ee.Feature(dry_cropland18, {'class': 1}),
  ee.Feature(dry_cropland19, {'class': 1}),
  ee.Feature(dry_cropland20, {'class': 1}),
  ee.Feature(dry_cropland21, {'class': 1}),
  ee.Feature(dry_cropland22, {'class': 1}),
  ee.Feature(dry_cropland23, {'class': 1}),
  ee.Feature(dry_cropland24, {'class': 1}),
  ee.Feature(dry_cropland25, {'class': 1}),
  ee.Feature(dry_cropland26, {'class': 1}),
  ee.Feature(dry_cropland27, {'class': 1}),
  ee.Feature(dry_cropland28, {'class': 1}),
  ee.Feature(dry_cropland29, {'class': 1}),
  ee.Feature(dry_cropland30, {'class': 1}),
  ee.Feature(dry_cropland31, {'class': 1}),
  ee.Feature(dry_cropland32, {'class': 1}),
  ee.Feature(dry_cropland33, {'class': 1}),
  ee.Feature(dry_cropland34, {'class': 1}),
  ee.Feature(dry_cropland35, {'class': 1}),
  
  ee.Feature(bareground1, {'class': 1}),
  ee.Feature(bareground2, {'class': 1}),
  ee.Feature(bareground3, {'class': 1}),
  ee.Feature(bareground4, {'class': 1}),
  ee.Feature(bareground5, {'class': 1}),
  ee.Feature(bareground6, {'class': 1}),
  ee.Feature(bareground7, {'class': 1}),
  ee.Feature(bareground8, {'class': 1}),
  ee.Feature(bareground9, {'class': 1}),
  ee.Feature(bareground10, {'class':1}),
  ee.Feature(bareground11, {'class':1}),
  ee.Feature(bareground12, {'class': 1}), 
  ee.Feature(bareground13, {'class': 1}),
  ee.Feature(bareground14, {'class': 1}),
  ee.Feature(bareground15, {'class': 1}),
  ee.Feature(bareground16, {'class': 1}),
  ee.Feature(bareground17, {'class': 1}),
  ee.Feature(bareground18, {'class': 1}),
  ee.Feature(bareground19, {'class': 1}),
  ee.Feature(bareground20, {'class': 1}),

  ee.Feature(urban_area1, {'class': 1}),
  ee.Feature(urban_area2, {'class': 1}),
  ee.Feature(urban_area3, {'class': 1}),
  ee.Feature(urban_area4, {'class': 1}),
  ee.Feature(urban_area5, {'class': 1}),
  ee.Feature(urban_area6, {'class': 1}),
  ee.Feature(urban_area7, {'class': 1}),
  ee.Feature(urban_area8, {'class': 1}),
  ee.Feature(urban_area9, {'class': 1}),
  ee.Feature(urban_area10, {'class': 1}),
  ee.Feature(urban_area11, {'class': 1}),
  ee.Feature(urban_area12, {'class': 1}),
  ee.Feature(urban_area13, {'class': 1}),
  ee.Feature(urban_area14, {'class': 1}),
  ee.Feature(urban_area15, {'class': 1}),
  ee.Feature(urban_area16, {'class': 1}),
  ee.Feature(urban_area17, {'class': 1}),
  ee.Feature(urban_area18, {'class': 1}),
  ee.Feature(urban_area19, {'class': 1}),
  ee.Feature(urban_area20, {'class': 1}),
  ee.Feature(urban_area21, {'class': 1}),
  ee.Feature(urban_area22, {'class': 1}),
  ee.Feature(urban_area23, {'class': 1}),
  ee.Feature(urban_area24, {'class': 1}),
  ee.Feature(urban_area25, {'class': 1}),
  ee.Feature(urban_area26, {'class': 1}),
  ee.Feature(urban_area27, {'class': 1}),
  ee.Feature(urban_area28, {'class': 1}),
  ee.Feature(urban_area29, {'class': 1}),
  ee.Feature(urban_area30, {'class': 1}),
  ee.Feature(urban_area31, {'class': 1}),
  ee.Feature(urban_area32, {'class': 1}),
  ee.Feature(urban_area33, {'class': 1}),
  ee.Feature(urban_area34, {'class': 1}),
    
  ee.Feature(wetland1, {'class': 1}),
  ee.Feature(wetland2, {'class': 1}),
  ee.Feature(wetland3, {'class': 1}),
  ee.Feature(wetland4, {'class': 1}),
  ee.Feature(wetland5, {'class': 1}),
  ee.Feature(wetland6, {'class': 1}),
  ee.Feature(wetland7, {'class': 1}),
  ee.Feature(wetland8, {'class': 1}),
  ee.Feature(wetland9, {'class': 1}),
  ee.Feature(wetland10, {'class': 1}),
  ee.Feature(wetland11, {'class': 1}),
  ee.Feature(wetland12, {'class': 1}),
  ee.Feature(wetland13, {'class': 1}),
  ee.Feature(wetland14, {'class': 1}),
  ee.Feature(wetland15, {'class': 1}),
  ee.Feature(wetland16, {'class': 1}),
  ee.Feature(wetland17, {'class': 1}),
  ee.Feature(wetland18, {'class': 1}),
  ee.Feature(wetland19, {'class': 1}),
  ee.Feature(wetland20, {'class': 1}),
  ee.Feature(wetland21, {'class': 1}),

  ee.Feature(green_veg1, {'class': 1}),
  ee.Feature(green_veg2, {'class': 1}),
  ee.Feature(green_veg3, {'class': 1}),
  ee.Feature(green_veg4, {'class': 1}),
  ee.Feature(green_veg5, {'class': 1}),
  ee.Feature(green_veg6, {'class': 1}),
  ee.Feature(green_veg7, {'class': 1}),
  ee.Feature(green_veg8, {'class': 1}),
  ee.Feature(green_veg9, {'class': 1}),
  ee.Feature(green_veg10, {'class': 1}),
  ee.Feature(green_veg11, {'class': 1}),
  ee.Feature(green_veg12, {'class': 1}),
  ee.Feature(green_veg13, {'class': 1}),
  ee.Feature(green_veg14, {'class': 1}),
  ee.Feature(green_veg15, {'class': 1}),
  ee.Feature(green_veg16, {'class': 1}),
  ee.Feature(green_veg17, {'class': 1}),
  ee.Feature(green_veg18, {'class': 1}),
  ee.Feature(green_veg19, {'class': 1}),
  ee.Feature(green_veg20, {'class': 1}),
  ee.Feature(green_veg21, {'class': 1}),
  ee.Feature(green_veg22, {'class': 1}), 
  ee.Feature(green_veg23, {'class': 1}),
  ee.Feature(green_veg24, {'class': 1}),
  ee.Feature(green_veg25, {'class': 1}),
  ee.Feature(green_veg26, {'class': 1}),
  ee.Feature(green_veg27, {'class': 1}),
  ee.Feature(green_veg28, {'class': 1}),
  ee.Feature(green_veg29, {'class': 1}),
  ee.Feature(green_veg30, {'class': 1}),
  ee.Feature(green_veg31, {'class': 1}),
  ee.Feature(green_veg32, {'class': 1}),
  ee.Feature(green_veg33, {'class': 1}),
  ee.Feature(green_veg34, {'class': 1}),
  ee.Feature(green_veg35, {'class': 1}),
])

classProperty = 'class'

# Get the values for all pixels in each polygon in the training.
training_dry = image_dry.select(bands).sampleRegions(collection = drypolygons, properties = ['class'], scale = 10)


# Create an SVM classifier with custom parameters.
classifier_dry = ee.Classifier.smileCart().train(**{
  'features': training_dry,
  'classProperty': classProperty,
})

# Classify the image.
classified_dry = image_dry.classify(classifier_dry)

# Accuracy assessment
# random uniforms to the training dataset.
withRandom = training_dry.randomColumn('random')

# We want to reserve some of the data for testing, to avoid overfitting the model.
split = 0.7;  # Roughly 70% training, 30% testing.
trainingPartition = withRandom.filter(ee.Filter.lt('random', split))
testingPartition = withRandom.filter(ee.Filter.gte('random', split))

# Trained with 70% of our data.
trainedClassifier = ee.Classifier.smileRandomForest(8).train(**{
  'features': trainingPartition,
  'classProperty': classProperty,
  'inputProperties': bands
})

# Classify the test FeatureCollection.
test = testingPartition.classify(trainedClassifier)

# Print the confusion matrix.
confusionMatrix = test.errorMatrix(classProperty, 'classification')
# print('Confusion Matrix', confusionMatrix.array())
# print('Overall Accuracy:',confusionMatrix.accuracy())

Post-classification Change Detection

In [ ]:
# Imoprt classified images from Drive to API
# Generate post-classification difference map
# Need to devide the thresholding of outcomes of MaxEnt Algorithm, and transform them into 0,1 classes
classified_wet = ee.Image("users/dylanyang/post_classification_change_class_wet").select('b1').clip(roi)
classified_dry = ee.Image("users/dylanyang/post_classification_change_class_dry").select('b1').clip(roi)

Map.addLayer(classified_wet, {}, 'img_wet')
Map.addLayer(classified_dry, {}, 'img_dry')

vis_params = {'bands': ['b1'], 'palette': ['black', 'gray', 'red'], 'min': 0, 'max': 2, 'opacity': 1}

# Difference Map: ratio
Class_Diff_ratio = classified_wet.divide(classified_dry) #(Difference Map)

Map.addLayer(classified_wet, vis_params, 'post classification wet')
Map.addLayer(classified_dry, vis_params, 'post classification dry')
Map.addLayer(Class_Diff_ratio, {'bands': ['b1'], 'palette': ['blue', 'gray', 'red'], 'min': 0, 'max': 2, 'opacity': 1.0}, 'Class_Diff')
Map

Area Calculation for Wet and Dry seasons (post-classified)

In [ ]:
areaImage = ee.Image.pixelArea().addBands(
     classified_wet)

areas_dic = areaImage.reduceRegion(**{
    'reducer': ee.Reducer.sum().group(**{
        'groupField': 1,
        'groupName': 'class'
    }),
    'geometry': roi,
    'scale': 10,
    'maxPixels': 1e10
    }) 
areas = areas_dic.getInfo()

classAreas = areas.get('groups')
i = 0

for item in classAreas:
    if i == 0:
        cls = 'Unclassified'
    elif i == 1:
        cls = 'Nonwater'
    else:
        cls = 'Water'
#     print('Wet Season Class{} area'.format(classAreas[i]['class']), item['sum']/1e6)
    print('Wet Season {} area'.format(cls), item['sum']/1e6)
    i += 1
#print('areas_dic:', areas_dic)

#print('areaimage', areaImage.getInfo())

#classified dry season image test
areaImage = ee.Image.pixelArea().addBands(
      classified_dry)

areas_dic = areaImage.reduceRegion(**{
    'reducer': ee.Reducer.sum().group(**{
        'groupField': 1,
        'groupName': 'class'
    }),
    'geometry': roi,
    'scale': 10,
    'maxPixels': 1e10
    }) 
areas = areas_dic.getInfo()

classAreas = areas.get('groups')
i = 0
for item in classAreas:
    if i == 0:
        cls = 'Unclassified'
    elif i == 1:
        cls = 'Nonwater'
    else:
        cls = 'Water'
    print('Dry Season {} area'.format(cls), item['sum']/1e6)
    i += 1


# Calculate the Difference between wet and dry seasons with different methods (Supervised Classification & Band Math)

In [ ]:
Diff_wet = classified_wet.divide(water_wet.add(1))
Diff_dry = classified_dry.divide(water_dry.add(1))
vis_params = {'bands': ['b1'], 'palette': ['black', 'gray', 'red'], 'min': 0, 'max': 2, 'opacity': 1}
Map.addLayer(Diff_wet, vis_params, 'Diff wet')
Map.addLayer(Diff_dry, vis_params, 'Diff dry')
Map

Note: 'classified_wet' / 'classified_dry' - Post-Classification; 'water_wet' / 'water_dry' - Band Math

In [ ]:
# Wet season' images
areaImage = ee.Image.pixelArea().addBands(
     Diff_wet)

areas_dic = areaImage.reduceRegion(**{
    'reducer': ee.Reducer.sum().group(**{
        'groupField': 1,
        'groupName': 'class'
    }),
    'geometry': roi,
    'scale': 10,
    'maxPixels': 1e10
    }) 
areas = areas_dic.getInfo()

classAreas = areas.get('groups')
i = 0

for item in classAreas:
    if i == 0:
        cls = 'Unclassified'
    elif i == 0.5:
        cls = 'From Nonwater to Water'
    elif i == 1:
        cls = 'Unchanged'
    else: 
        cls = 'From Water to Nonwater'
    print('Wet Season {} area'.format(cls), item['sum']/1e6)
    i += 1
    
# Dry season images
areaImage = ee.Image.pixelArea().addBands(
     Diff_dry)

areas_dic = areaImage.reduceRegion(**{
    'reducer': ee.Reducer.sum().group(**{
        'groupField': 1,
        'groupName': 'class'
    }),
    'geometry': roi,
    'scale': 10,
    'maxPixels': 1e10
    }) 
areas = areas_dic.getInfo()

classAreas = areas.get('groups')
i = 0

for item in classAreas:
    if i == 0:
        cls = 'Unclassified'
    elif i == 0.5:
        cls = 'From Nonwater to Water'
    elif i == 1:
        cls = 'Unchanged'
    else: 
        cls = 'From Water to Nonwater'
    print('Dry Season {} area'.format(cls), item['sum']/1e6)
    i += 1

# Data Export

In [ ]:
# #export roi(polygons) to Drive
# geemap.ee_to_shp(wetpolygons, filename='aoi_polygons/data/wet_polygons.shp')

In [ ]:
# #export polygons to Drive
# geemap.ee_to_shp(polygons, filename='aoi_polygons/data/poly_dry.shp')

In [ ]:
# #export classified wet season image to Drive
# geemap.ee_export_image_to_drive(classified_wet, description='post_classification_change_class_wet', folder='classification', region=roi, scale=10)

In [ ]:
# #export classified dry season image to Drive
# geemap.ee_export_image_to_drive(classified_dry, description='post_classification_change_class_dry', folder='classification', region=roi, scale=10)

In [ ]:
# #export classified difference map to Drive

# geemap.ee_export_image_to_drive(Diff_wet, description='Wet Difference Map_change_class', folder='classification', region=roi, scale=10)
# geemap.ee_export_image_to_drive(Diff_dry, description='Dry Difference Map_change_class', folder='classification', region=roi, scale=10)

In [ ]:
# #export post-classification difference map to Drive

# geemap.ee_export_image_to_drive(Class_Diff_ratio, description='Ratio Post-class Difference Map', folder='classification', region=roi, scale=10)